In [37]:
#%pylab notebook

#Pandas, numpy & Dates
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from numpy import arange

#Working directory tools
import os
from os import listdir
cwd = os.path.abspath('../')

#Max display (monitor)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>")) #75%

#Load custom functions
exec(open(cwd + "\\Scripts\\Functions.py").read())

#Stats (skewness and kurtosis)
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis

from scipy import stats
from scipy import integrate

#Widgets & plots (as widgets)
import ipywidgets as widgets
from ipywidgets import ToggleButtons, VBox, HBox, HTML

from bqplot.interacts import (FastIntervalSelector, 
                              IndexSelector, BrushIntervalSelector,
                              BrushSelector, MultiSelector, 
                              LassoSelector, PanZoom, HandDraw)

#interaction
from traitlets import link
from IPython.display import HTML, display, clear_output

#File exploring window
from tkinter import Tk, filedialog 
from tkinter import *

from bqplot import *

<a id='TableOfContents'></a>
<center> <h1 style = 'background-color:#D0C8D6; color:black; height:35px; padding: 5px;'> Table of Contents</h1> </center> 

### [Introduction](#First)

### [Can We Look Back for Comparable Cases?](#Second)

### [A Closer Look at Each Stage](#Third)

### [What happened with Consecutive Spreads Since The Christmas Rally?](#Fourth)

### [Useful Questions and (Some) Answers](#Fifth)


<a id='First'></a>
[<center> <h1 style = 'background-color:#D0C8D6; color:black; height:35px; padding: 5px;'> Introduction </h1> </center>](#TableOfContents)

## Objective 

Recent US financial market developements have caused the market to begin pricing a potential recession by 2019-end and thus, a change in Fed Monetary Policy towards a more accommodative approach, including rate cuts and stopping balance sheet normalization by Q3 of 2019. The purpose of this small document is to review current consecutive-hike spread (or Pc-Bias) strategies, involving Quarterly and a Non-Quarterly meetings, to determine wether we can trade them successfully on the current and future potential curve environment. 

## Some brief background on "Press Conference Bias"

What is the Press Conference Bias? Simply put, a bias for monetary policy action towards quarterly meetings (March, June, September and December meetings), this bias is currently taken into account by the market when pricing rate moves. 

To find out when this bias was born, we first have to go back to 2011, when on April 27th, Fed Chair Bernanke gave the first post-FOMC meeting press conference in the history of the Federal Reserve (you can see it [here]). This new monetary policy communication tool was not scheduled regularly at the beginning, therefore the Fed Chair would appear before the press whenever there was a need to communicate with the public. As the Fed continued polishing its communication tools, and forward guidance gained traction, by 2013, these press conferences became regularly held on quarterly meetings. 

FOMC members individual projections were also released to the public only on press conference meetings, so this schedule eventually gave the market the idea that quarterly meetings were more relevant for monetary policy action, and thus the "press conference bias" was eventually fully priced in by the market by the second half of 2014. As the most recent rate-hike cycle began, the bias was confirmed by the Fed, as every single rate move from 2015 through 2018 was done on a quarterly meeting. 

On June 13th of 2018, Fed Chair Jerome Powell announced that the Fed would be holding a press conference on every meeting. He made sure to remark that the decision was only aimed at improving communication, and that it had no other implications regarding monetary policy, so the market reaction over time was to continue pricing the bias. 

[here]: https://www.federalreserve.gov/monetarypolicy/fomcpresconf20110427.htm

## Some Considerations

We are not going to look at consecutive spreads, or flies, broadly, but only focusing on a specific set of them. Here we outline the conditions a fly has to meet to be considered into this analysis: 

>- **Shorts only (recommended):** We are exclusively looking to consider spreads were we can short the non-quarterly meeting vs the adjacent quarterly meeting. 
>
> 
>- **Fixed structure:** Non-Quarterly meeting as the first leg always, followed by the Quarterly meeting afterwards.
>
>
>- **No December:** Excluding flies that have any December hike as one of their legs.
>
>
>- **2020 Spreads (recommended):** Focusing on 2020 flies, where the curve is the smoothest.

## Why look at these strategies now?

The idea behind these strategies is that, after the rate crash and curve flattening that proceeded the shift in FedSpeak back in November 2018, the curve has smoothened somewhat and that it might smoothen even further, particularly on the back end (2020). 

Under this current curve environment, the market is pricing a rate-cut cycle, with cuts seemingly being priced on every consecutive meeting, smoothing out the curve as a consequence. This smooth curve hypothesis, wether it has a positive slope with rate cuts or is completely flat with no rate moves priced in, favours this particular set of flies, as they can perform on the following potential scenarios: 

>- Mean reversion around a flat curve.
>
>
>- Positive (smooth) slope with rate cuts being priced consecutively (with no PC-bias), level independent. 
>
>
>- Monetary policy shift back to rate hikes, with PC-bias.

Of course these spreads can fail, here are some of the potential scenarios where they can underperform and break down:

>- Positive (non-smooth) slope with rate cuts being priced predominantly on quarterly meetings (PC-Bias). 
>
>
>- Monetary policy shift back to rate hikes, without PC-Bias (smooth negative sloped curve).

<a id='Second'></a>
[<center> <h1 style = 'background-color:#D0C8D6; color:black; height:35px; padding: 5px;'> Can We Look Back for Comparable Cases?</h1> </center>](#TableOfContents)

## Hikes Curve Progression and Stages 

We go back to 2012 and look at the evolution of the hikes curve from year to year through 2019 looking for comparable cases. Remember that post-FOMC Meeting press conferences were already introduced by 2012, fixed on quarterly meetings by 2013, and fully priced by the market by the second half of 2014. 

We can then split this period into three stages (arbitrarily). The reasoning behind this split is to allow us to better find comparable cases for these types of strategies and to determine under which scenarios we should, and shouldn't, trade them.

In [38]:
#Plot data
############################################################  
Summary = FFsDataList['Hikes_Generic_Data']['2012'].describe() 

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure8 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure8.title =  'Figure 1: ' + 'Average Hikes Curve: 2012'

#Figure layout (except size attributes, defined on section 4) 
figure8.title_style = {'font-size': '13px'}
figure8.background_style = {'fill': 'white'} 

figure8.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure8.layout.width = '30%'
figure8.layout.height = '330px'
figure8.legend_location = 'top-left'
figure8.layout.left = '30px'


############################################################ 
############################################################ 
############################################################ 


#Plot data
############################################################ 
Summary = FFsDataList['Hikes_Generic_Data']['2013'].describe() 

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure9 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure9.title =  'Figure 2: ' + 'Average Hikes Curve: 2013'

#Figure layout (except size attributes, defined on section 4) 
figure9.title_style = {'font-size': '13px'}
figure9.background_style = {'fill': 'white'} 

figure9.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure9.layout.width = '30%'
figure9.layout.height = '330px'
figure9.legend_location = 'top-left'
figure9.layout.left = '60px'

############################################################ 
############################################################ 
############################################################ 



#Plot data
############################################################
Summary = FFsDataList['Hikes_Generic_Data']['2014/01':'2014/06'].describe() 

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure10 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure10.title = 'Figure 3: ' + 'Average Hikes Curve: 2014 H1'

#Figure layout (except size attributes, defined on section 4) 
figure10.title_style = {'font-size': '13px'}
figure10.background_style = {'fill': 'white'} 

figure10.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure10.layout.width = '30%'
figure10.layout.height = '330px'
figure10.legend_location = 'top-left'
figure10.layout.left = '90px'

In [39]:
#Plot data
############################################################   
Summary = FFsDataList['Hikes_Generic_Data']['2014/07':'2014/12'].describe()  

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure11 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure11.title =  'Figure 4: ' + 'Average Hikes Curve: 2014 H2'

#Figure layout (except size attributes, defined on section 4) 
figure11.title_style = {'font-size': '13px'}
figure11.background_style = {'fill': 'white'} 

figure11.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure11.layout.width = '30%'
figure11.layout.height = '330px'
figure11.legend_location = 'top-left'
figure11.layout.left = '30px'


############################################################
############################################################
############################################################


#Plot data
############################################################  
Summary = FFsDataList['Hikes_Generic_Data']['2015'].describe()

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure12 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure12.title =  'Figure 5: ' + 'Average Hikes Curve: 2015'

#Figure layout (except size attributes, defined on section 4) 
figure12.title_style = {'font-size': '13px'}
figure12.background_style = {'fill': 'white'} 

figure12.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure12.layout.width = '30%'
figure12.layout.height = '330px'
figure12.legend_location = 'top-left'
figure12.layout.left = '60px'

############################################################
############################################################
############################################################


#Plot data
############################################################   
Summary = FFsDataList['Hikes_Generic_Data']['2016'].describe()   

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure13 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure13.title = 'Figure 6: ' + 'Average Hikes Curve: 2016'

#Figure layout (except size attributes, defined on section 4) 
figure13.title_style = {'font-size': '13px'}
figure13.background_style = {'fill': 'white'} 

figure13.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure13.layout.width = '30%'
figure13.layout.height = '330px'
figure13.legend_location = 'top-left'
figure13.layout.left = '90px'

In [40]:
#Plot data
############################################################  
Summary = FFsDataList['Hikes_Generic_Data']['2017'].describe()    

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure14 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure14.title =  'Figure 7: ' + 'Average Hikes Curve: 2017'

#Figure layout (except size attributes, defined on section 4) 
figure14.title_style = {'font-size': '13px'}
figure14.background_style = {'fill': 'white'} 

figure14.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure14.layout.width = '30%'
figure14.layout.height = '330px'
figure14.legend_location = 'top-left'
figure14.layout.left = '30px'


############################################################
############################################################
############################################################


#Plot data
############################################################ 
Summary = FFsDataList['Hikes_Generic_Data']['2018'].describe()  

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure15 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure15.title = 'Figure 8: ' + 'Average Hikes Curve: 2018'

#Figure layout (except size attributes, defined on section 4) 
figure15.title_style = {'font-size': '13px'}
figure15.background_style = {'fill': 'white'} 

figure15.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure15.layout.width = '30%'
figure15.layout.height = '330px'
figure15.legend_location = 'top-left'
figure15.layout.left = '60px'


############################################################  
############################################################  
############################################################  


#Plot data
############################################################   
Summary = FFsDataList['Hikes_Generic_Data']['2019'].describe() 

#Average curve +- 1 std
AverageCurve = np.array(Summary.loc['mean'])  
Minus1Std = np.array(Summary.loc['mean'] - Summary.loc['std']) 
Plus1Std = np.array(Summary.loc['mean'] + Summary.loc['std']) 

#Tickers
Scatt_x_data = ['H' + str(i) for i in range(1, len(AverageCurve) + 1)]
Scatt_y_data = AverageCurve
############################################################ 

#Step 1: Scales & Axes
############################################################ 
#Scales 
Scatt_x = OrdinalScale()
Scatt_y = LinearScale()

#X-Axis
ax_x = Axis(scale = Scatt_x, 
            tick_format = '', 
            label_location = 'end', 
            label = 'Hike',
            num_ticks = 8,
            tick_style = {'font-size': '11px'}, 
            label_offset = '40px')

#Y-Axis
ax_y = Axis(scale = Scatt_y, 
            orientation = 'vertical', 
            tick_format = '.1f', 
            label_location = 'end',
            label = 'bps', 
            grid_color = '#EDEDED',
            tick_style = {'font-size': '12px'}, 
            label_offset = '40px')
############################################################

#Step 2: Marks (plots)
############################################################     
#Marks, scatter plot (legs)
Scatt = Scatter(x = Scatt_x_data, 
                y = Scatt_y_data,               
                scales = {'x': Scatt_x, 
                          'y': Scatt_y},
                colors = ['#01B0F0'],
                default_size = 80,
                default_opacities = [0.7], 
                stroke='black', 
                unhovered_style={'opacity': 0.2})

#Curve interpolation (mean curve)
lin = Lines(x = Scatt.x, y = Scatt.y, 
            scales = {'x': Scatt_x, 'y' : Scatt_y},
            colors = ['#01B0F0'], 
            stroke_width = 2, interpolation = 'cardinal')

#Curve interpolation (+- 1 std)
lin2 = Lines(x = Scatt.x, y = [Minus1Std, Plus1Std], 
            scales = {'x': Scatt_x, 'y' : Scatt_y}, 
            colors = ['#DE2369'], 
            stroke_width = 2.5, 
            interpolation = 'cardinal', 
            fill = 'between', 
            fill_colors = ['#0070C0'], 
            fill_opacities = ['0.1'])
############################################################

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['', '.2f'], 
                 labels = ['Hike', 'Price'])

#Set the tooltip interaction(s)
Scatt.tooltip = def_tt
Scatt.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################
#initial Figure
figure16 = Figure(marks = [lin, lin2, Scatt], 
                 axes = [ax_x, ax_y])

#Figure Title
figure16.title = 'Figure 9: ' + 'Average Hikes Curve: 2019'

#Figure layout (except size attributes, defined on section 4) 
figure16.title_style = {'font-size': '13px'}
figure16.background_style = {'fill': 'white'} 

figure16.fig_margin = {'top':40, 'bottom':50, 'left':50, 'right':10}
figure16.layout.width = '30%'
figure16.layout.height = '330px'
figure16.legend_location = 'top-left'
figure16.layout.left = '90px'

In [41]:
VBox([widgets.HTML('<a id="Figure_9"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b> Expected Hikes Curve by Year: </b>' + 
                   'Average expected hikes curve for each year plus/minus one standard deviation, split by sub-period (rows).' +
                   ' From 2012 through 2019. </p>'),
      BorderHtmlW, VBox([HBox([figure8, figure9, figure10]), 
                         HBox([figure11, figure12, figure13]), 
                         HBox([figure14, figure15, figure16])]), BorderHtmlW])

We can summarize the three stages as follows:

>- **Pre-Hiking Cycle:** From 2012 through the first half of 2014.This stage was characterized by a smooth positive slope with hike expectations in the back-end, low hike volatility in the front and no Press Conference Bias.
>
>
>- **Transition to Full Hiking Cycle:** From the second half of 2014 through 2016.This stage was characterized by a Press Conference Bias seasonality fully priced, a flatter slope, transitioning to a negative slope with hike expectations in the front-end, and higher hike volatility (particularly in the front-end as well).
>
>
>- **Full-Hiking Cycle:** From 2017 through 2018.This stage was characterized by a curve that transitioned into a full rate hike cycle, with a smooth negative slope with hike expectations in the back-end, press conference bias and higher hike volatility in the front-end.
>
>

## Lets take a first look at the full set of comparable spreads

We now go a bit deeper, and take a closer look at the specific set of spreads that meet our criteria for the full period. The idea behind this is to take a quick glance at how they performed and whether each one of the sub-periods defined arbitrarily were actually associated with different behaviours for these spreads, as the curve changed shapes. 

[`Figure 10`](#Figure_10) below is a frequency heatmap, pretty useful for comparing multiple distributions quickly. When looking at this plot you basically see each spread's distribution (histogram), from above. Spreads are plotted on the X-axis and histogram price intervals (bps) are plotted on the Y-axis, while the number of observations per interval is represented on a color scale. The sample is the last 180 trading days to maturity for every spread:

In [42]:
#Set of hike spreads 
Spreads_set = ['F12H12', 'J12M12', 'Q12U12', 
               'F13H13', 'K13M13', 'N13U13', 
               'F14H14', 'J14M14', 'N14U14', 
               'F15H15', 'J15M15', 'N15U15', 
               'F16H16', 'J16M16', 'N16U16', 
               'G17H17', 'K17M17', 'N17U17', 
               'F18H18', 'K18M18', 'Q18U18', 
               'F19H19', 'K19M19', 'Q19U19', 
               'F20H20', 'J20M20', 'N20U20']

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-90:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
#Compute histogram (bins for all aggregated data)
FullBins = np.histogram(np.reshape(np.array(GridFullData.T), 
                                   (1, 90*len(Spreads_set))))

#Compute individual counts for each spread (same bins for all data)
CountDf =  pd.DataFrame(columns = Spreads_set)

for i in range(0, len(Spreads_set)):
    
    CountDf.iloc[:, i] = np.histogram(np.array(
                         GridFullData.iloc[:, i]), bins = FullBins[1])[0]


############################################################    
############################################################
############################################################

    
#Plot data
############################################################
x_data = Spreads_set
y_data = FullBins[1]
MapData = np.array(CountDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
y_sc = LinearScale()
x_sc = OrdinalScale()
col_sc = ColorScale(min = float(np.min(MapData)), 
                    max = float(np.max(MapData)), 
    colors = ['#EDEDED', '#01B0F0'])

ax_y = Axis(scale = x_sc, 
            label_location = 'end',
            label_offset = '40px',
            #num_ticks = 9,
            tick_style = {'font-size': '9px'}) #10px

ax_x = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label_location = 'middle',
            label = 'Price (bps)',
            label_offset = '30px', 
            num_ticks = len(y_data))

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Count', 
                   tick_format = '0f', 
                   label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = y_data, 
                       column = x_data, 
                       color = MapData.T, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['column', 'color'], 
                 formats = ['', '0f'], 
                 labels = ['Hike Spread', 
                           'Count'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y, col_ax])

#Figure Title
figure7.title = 'Figure 10: ' +  'Consecutive Hike Spreads Frequency HeatMap'

#Figure layout (except size attributes, defined on section 4)
figure7.title_style = {'font-size': '13px'}
figure7.background_style = {'fill': 'white'}

figure7.fig_margin = {'top':40, 'bottom':90, 'left':40, 'right':30}
figure7.layout.width = '95%'#'35%'
figure7.layout.left = '30px'
figure7.layout.height = '500px'

In [43]:
VBox([widgets.HTML('<a id="Figure_10"></a>'), 
      widgets.HTML('<p style = "color: #125CB2;"> <b> First Look at Comparable Spreads: </b>' + 
                   'Consecutive Hike Spreads Frequency HeatMap for all' + 
                   ' relevant spreads through all stages under analysis (2012-2020).' +
                   ' Spreads with December hikes are excluded. </p>'),
      BorderHtmlW, figure7, BorderHtmlW])

We can see clearly that each stage curve shapes translate to different spread's behaviour, as we should expect. Also this gives us an idea regarding under which scenario these trades are more comfortable, and tradable. Here are some initial observations:

>- With no seasonal effects and a positive sloped curve spreads tend to trade around zero, without significant volatility the flatter the curve gets. This is the kind of scenario where we can make use of consecutive spreads alongside jumping spreads between the same type of meetings, increasing our options and the potential exploiting of particular meeting mispricings.
>
>
>- When seasonal effects become significant consecutive spreads break down, trading almost always below zero, with huge volatility and completely dependant on the quarterly meeting leg hike expectations. A curve with cuts and seasonal patters priced in would behave similarly but with spreads trading above zero. This is the kind of scenario where we would prefer to avoid trading consecutive spreads.
>
>
>- If we look at current spreads (F19H19 to N20U20) with the data we have so far, we can see that we are moving back to a transition and pre-hiking cycle kind of curves.

<a id='Third'></a>
[<center> <h1 style = 'background-color:#D0C8D6;color:black;font-size:30px;'> A Closer Look at Each Stage</h1> </center>](#TableOfContents)

## How Are We Going to Do It?

For each stage: First we look at consecutive spreads, then we look at jumping spreads (between quarterly meetings only) and we finally compare the two. The reason behind looking at jumping spreads between quarterly meetings is due to the fact that whenever Press Conference bias is present on the curve, these tend to be the most tradable spreads, while consecutive spreads become extremely directional. 

We will be using frequency heatmaps, fixed day to maturity time series, aggregated spread data histograms to compare consecutive and jumping spreads and look at their distributions across all stages, and aggregated spread data fixed day to maturity time series:

>- **Frequency HeatMaps:** Remember these are pretty useful for comparing multiple distributions quickly. Spreads are plotted on the X-axis and histogram price intervals on the Y-axis, the number of observations per interval is represented on a color scale.
>
>
>- **Fixed Day to Maturity Time Series:** The idea behind using these plots is to take a closer look at how each spread behaved 180 trading days before their expiration and see if there is any pattern (Rollup or Rolldown) as they approach expiration.
> 
>
>- **Aggregated Data Histograms:** We aggregate all of our 180 days to maturity data (all of the spreads) and plot their distribution, we do this for consecutive and jumping spreads separatedly. The idea is to see if there is any difference between them, we want to look at their distribution's shapes, their tails, see if they are centered around different means, etc. 
>
>
>- **Aggregated Data Day to Maturity Time Series:** This is simply an aggregation of individual spreads day to maturity time series, we do it by creating an equally weighted "portfolio" of all consecutive spreads and repeat the same process for jumping spreads, its a summary of what we saw individually for each spread on their corresponding day to maturity time series. 

## Pre-Hiking Cycle (2012-2014)

The smooth positive slope and lack of PC-bias on this period meant that shorting hike spreads (consecutive or jumping) was very comfortable. Their fair price was close to zero the flatter the curve got, mispricings were small and volatility was low as well. This is the kind of curve scenario were these spreads behave the best, with no big shocks, mean reversion and favorable rolldown with hikes expiring near or below zero. 

The question that arises on these type of scenarios where the slope is smooth is wether there is a difference between consecutive spreads and jumping spreads (between quarterly or non quarterly meetings). With seasonal effects present, we discard consecutive spreads as the hedging becomes ineffective but when the curve is smooth we find that we have a whole new set of spreads at our disposal. 

In [44]:
#Title 
Title = 'Pre Hiking-Cycle'

#Set of hike spreads 
Spreads_set = ['F12H12', 'J12M12', 'Q12U12', 
               'F13H13', 'K13M13', 'N13U13', 
               'F14H14', 'J14M14', 'N14U14']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
#Compute equally weighted portfolio of consecutive spreads
Portfolio_Consec = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                              len(GridFullData.columns)))
    
#Compute histogram (bins for all aggregated data)
FullDataArray_Consec = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))

FullBins = np.histogram(FullDataArray_Consec)

#Store bins for scatter
FullBins_Consec = FullBins


#Compute individual counts for each spread (same bins for all data)
CountDf =  pd.DataFrame(columns = Spreads_set)

for i in range(0, len(Spreads_set)):
    
    CountDf.iloc[:, i] = np.histogram(np.array(
                         GridFullData.iloc[:, i]), bins = FullBins[1])[0]  


############################################################    
############################################################
############################################################

 
#Plot data
############################################################
x_data = Spreads_set
y_data = FullBins[1]
MapData = np.array(CountDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
y_sc = LinearScale()
x_sc = OrdinalScale()
col_sc = ColorScale(min = float(np.min(MapData)), 
                    max = float(np.max(MapData)), 
    colors = ['#EDEDED', '#DE2369'])

ax_y = Axis(scale = x_sc, 
            label_location = 'end',
            label_offset = '40px',
            tick_style = {'font-size': '10px'}) 

ax_x = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label_location = 'end',
            label = 'Price (bps)',
            label_offset = '40px', 
            num_ticks = len(y_data))

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Count', 
                   tick_format = '0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = y_data, 
                       column = x_data, 
                       color = MapData.T, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['column', 'color'], 
                 formats = ['', '0f'], 
                 labels = ['Hike Spread', 
                           'Count'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7_2 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y])

#Figure Title
figure7_2.title = ( 'Figure 11: ' +  
                   ' Consecutive Spreads Frequency Map' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_2.title_style = {'font-size': '13px'}
figure7_2.background_style = {'fill': 'white'}

figure7_2.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure7_2.layout.width = '40%'#'650px'#'35%'
figure7_2.layout.left = '10px'
figure7_2.layout.height = '420px'


############################################################    
############################################################
############################################################

 
#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure7_3 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure7_3.title = ( 'Figure 12: ' + 
                   ' Consecutive Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_3.title_style = {'font-size': '13px'}
figure7_3.background_style = {'fill': 'white'}
    
figure7_3.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure7_3.layout.width = '60%'
figure7_3.layout.height = '420px'
figure7_3.legend_location = 'bottom-left'

In [45]:
#Set of hike spreads 
Spreads_set = ['H12M12', 'M12U12', 'U12Z12', 
               'H13M13', 'M13U13', 'U13Z13',
               'H14M14', 'M14U14', 'U14Z14']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat

#Compute equally weighted portfolio of consecutive spreads
Portfolio_Jumping = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                               len(GridFullData.columns)))    
    
#Compute histogram (bins for all aggregated data)
FullDataArray_Jumping = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))

FullBins = np.histogram(FullDataArray_Jumping)


#Compute individual counts for each spread (same bins for all data)
CountDf =  pd.DataFrame(columns = Spreads_set)

for i in range(0, len(Spreads_set)):
    
    CountDf.iloc[:, i] = np.histogram(np.array(
                         GridFullData.iloc[:, i]), bins = FullBins[1])[0]  

    
############################################################
############################################################
############################################################
 
    
#Plot data
############################################################
x_data = Spreads_set
y_data = FullBins[1]
MapData = np.array(CountDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
y_sc = LinearScale()
x_sc = OrdinalScale()
col_sc = ColorScale(min = float(np.min(MapData)), 
                    max = float(np.max(MapData)), 
    colors = ['#EDEDED', '#01B0F0'])

ax_y = Axis(scale = x_sc, 
            label_location = 'end',
            label_offset = '40px',
            tick_style = {'font-size': '10px'}) 

ax_x = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label_location = 'end',
            label = 'Price (bps)',
            label_offset = '40px', 
            num_ticks = len(y_data))

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Count', 
                   tick_format = '0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = y_data, 
                       column = x_data, 
                       color = MapData.T, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['column', 'color'], 
                 formats = ['', '0f'], 
                 labels = ['Hike Spread', 
                           'Count'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7_4 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y])

#Figure Title
figure7_4.title =  ( 'Figure 13: ' +  
                    ' Jumping Spreads Frequency Map' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_4.title_style = {'font-size': '13px'}
figure7_4.background_style = {'fill': 'white'}

figure7_4.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure7_4.layout.width = '40%'#'650px'#'35%'
figure7_4.layout.left = '10px'
figure7_4.layout.height = '420px'

    
############################################################
############################################################
############################################################
 
    
#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure7_5 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure7_5.title =  ( 'Figure 14: ' + 
                    ' Jumping Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_5.title_style = {'font-size': '13px'}
figure7_5.background_style = {'fill': 'white'}
    
figure7_5.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure7_5.layout.width = '60%'
figure7_5.layout.height = '420px'
figure7_5.legend_location = 'bottom-left'

    
############################################################
############################################################
############################################################
 
    
#Plot data
############################################################
Title = 'Pre Hiking Cycle'
y_data = FullDataArray_Consec
y2_data = FullDataArray_Jumping
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '12px'},
                 label_location = 'end',
                 label = 'Price (bps)',
                 label_offset = '40px')

#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '0f',     
                 tick_style = {'font-size': '12px'}, 
                 grid_color = '#EDEDED',
                 label_location = 'middle',
                 label = 'Count',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['Consecutive'],
                display_legend = True)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.4] * HistPlot.bins


#Histogram plot (price)
HistPlot2 = Hist(sample = y2_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['Jumping'],
                display_legend = True)    

#Setting histogram bar opacities 
HistPlot2.opacities = [0.4] * HistPlot2.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['', '.2f'], 
         labels = ['Count', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}

#Set up the tooltip interaction (price)
HistPlot2.tooltip = def_tt
HistPlot2.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure7_6 = Figure(marks = [HistPlot, HistPlot2], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure7_6.title = ( 'Figure 23: ' +  Title + 
                   ' Spreads Aggregated Data' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_6.title_style = {'font-size': '13px'}
figure7_6.background_style = {'fill': 'white'} 

figure7_6.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure7_6.layout.width = '45%'
figure7_6.layout.left = '30px'
figure7_6.layout.height = '450px'
figure7_6.legend_location = 'top-left'

In [46]:
VBox([widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ': </b>' + 
                   'Closer look at Consecutive and Quarterly (Jumping) Hike Spreads. ' +  
                   ' Frequency heatmap and day to maturity time series,' +
                   ' 180 days before expiry. </p>'),
      BorderHtmlW, VBox([HBox([figure7_2, figure7_3]), 
                         HBox([figure7_4, figure7_5])]), BorderHtmlW]) 

We can clearly see that for this stage consecutive and jumping spreads behaved similarly, with a smooth slope and no PC-bias consecutive spreads become interesting alternatives for jumping (quarterly meetings) spreads. Consecutive spreads did display a higher volatility on average (higher daily standard deviation) and better rolldown, which might make them better for trading overal, although this might not be that relevant for our current environment. Here is a summary of observatios below:

<br>

|Consecutive Spreads|Jumping Spreads|
|-----------|-----------|
|Mean reverting around 0 <br> (flatter slopes on the front-end)|Mean reverting around 0 <br> (flatter slopes on the front-end)|
|Small trade ranges, ~2bp range and <br> an average daily std of ~0.14 basis points|Small trade ranges, ~2bp range and  <br> an average daily std of ~0.12 basis points|
|We can see a slight rolldown as <br> spreads moved closer to expiration.|No clear rolldown/rollup as <br> spreads moved closer to expiration.|

<br>

## Transition to Full Hiking Cycle (2015-2016)

On this period the PC-bias starts getting priced in, plus the curve starts to flatten as hike expectations move closer to the front. Higher front-end volatility and the potential for Non-quarterly meetings snapping above its quarterly pairs in the front-end make for harder-to-trade spreads, and even harder to hold due to the potential for the non-quarterly leg getting skewed once it gets closer to expiration. We begin seeing differences between consecutive and jumping spreads, as the PC-bias begins settling in. Also we experience for the first time Non quarterly meetings getting skewed in the front end, we will talk about this later. 

In [47]:
#Title 
Title = 'Transition to Full Hiking Cycle'

#Set of hike spreads 
Spreads_set = ['F15H15', 'J15M15', 'N15U15', 
               'F16H16', 'J16M16', 'N16U16']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
#Compute equally weighted portfolio of consecutive spreads
Portfolio_Consec = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                              len(GridFullData.columns)))
    
#Compute histogram (bins for all aggregated data)
FullDataArray_Consec = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))

FullBins = np.histogram(FullDataArray_Consec)

#Store bins for scatter
FullBins_Consec = FullBins


#Compute individual counts for each spread (same bins for all data)
CountDf =  pd.DataFrame(columns = Spreads_set)

for i in range(0, len(Spreads_set)):
    
    CountDf.iloc[:, i] = np.histogram(np.array(
                         GridFullData.iloc[:, i]), bins = FullBins[1])[0]
    
    
############################################################
############################################################
############################################################


#Plot data
############################################################
x_data = Spreads_set
y_data = FullBins[1]
MapData = np.array(CountDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
y_sc = LinearScale()
x_sc = OrdinalScale()
col_sc = ColorScale(min = float(np.min(MapData)), 
                    max = float(np.max(MapData)), 
    colors = ['#EDEDED', '#DE2369'])

ax_y = Axis(scale = x_sc, 
            label_location = 'end',
            label_offset = '40px',
            tick_style = {'font-size': '10px'}) 

ax_x = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label_location = 'middle',
            label = 'Price (bps)',
            label_offset = '40px', 
            num_ticks = len(y_data))

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Count', 
                   tick_format = '0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = y_data, 
                       column = x_data, 
                       color = MapData.T, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['column', 'color'], 
                 formats = ['', '0f'], 
                 labels = ['Hike Spread', 
                           'Count'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7_8 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y])

#Figure Title
figure7_8.title = ( 'Figure 15: ' + 
                   'Consecutive Spreads Frequency Map' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_8.title_style = {'font-size': '13px'}
figure7_8.background_style = {'fill': 'white'}

figure7_8.fig_margin = {'top':40, 'bottom':20, 'left':100, 'right':50}
figure7_8.layout.width = '35%'#'650px'#'35%'
figure7_8.layout.left = '10px'
figure7_8.layout.height = '395px'

    
############################################################
############################################################
############################################################


#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure7_9 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure7_9.title = ( 'Figure 16: ' +  
                   ' Consecutive Hike Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_9.title_style = {'font-size': '13px'}
figure7_9.background_style = {'fill': 'white'} 
    
figure7_9.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure7_9.layout.width = '60%'
figure7_9.layout.height = '420px'
figure7_9.legend_location = 'top-left'
figure7_9.layout.left = '20px'

In [48]:
#Set of hike spreads 
Spreads_set = ['H15M15', 'M15U15', 'U15Z15',
               'H16M16', 'M16U16', 'U16Z16']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat

#Compute equally weighted portfolio of consecutive spreads
Portfolio_Jumping = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                               len(GridFullData.columns)))    
    
#Compute histogram (bins for all aggregated data)
FullDataArray_Jumping = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))

FullBins = np.histogram(FullDataArray_Jumping)


#Compute individual counts for each spread (same bins for all data)
CountDf =  pd.DataFrame(columns = Spreads_set)

for i in range(0, len(Spreads_set)):
    
    CountDf.iloc[:, i] = np.histogram(np.array(
                         GridFullData.iloc[:, i]), bins = FullBins[1])[0]  

    
############################################################
############################################################
############################################################


#Plot data
############################################################
x_data = Spreads_set
y_data = FullBins[1]
MapData = np.array(CountDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
y_sc = LinearScale()
x_sc = OrdinalScale()
col_sc = ColorScale(min = float(np.min(MapData)), 
                    max = float(np.max(MapData)), 
    colors = ['#EDEDED', '#01B0F0'])

ax_y = Axis(scale = x_sc, 
            label_location = 'end',
            label_offset = '40px',
            tick_style = {'font-size': '10px'}) 

ax_x = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label_location = 'middle',
            label = 'Price (bps)',
            label_offset = '40px', 
            num_ticks = len(y_data))

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Count', 
                   tick_format = '0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = y_data, 
                       column = x_data, 
                       color = MapData.T, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['column', 'color'], 
                 formats = ['', '0f'], 
                 labels = ['Hike Spread', 
                           'Count'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7_10 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y])

#Figure Title
figure7_10.title =  ( 'Figure 17: ' + 
                     'Jumping Spreads Frequency Map' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_10.title_style = {'font-size': '13px'}
figure7_10.background_style = {'fill': 'white'}

figure7_10.fig_margin = {'top':40, 'bottom':20, 'left':100, 'right':50}
figure7_10.layout.width = '35%'#'650px'#'35%'
figure7_10.layout.left = '10px'
figure7_10.layout.height = '395px'

    
############################################################
############################################################
############################################################


#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure7_11 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure7_11.title =  ( 'Figure 18: '  
                     ' Jumping Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_11.title_style = {'font-size': '13px'}
figure7_11.background_style = {'fill': 'white'}
    
figure7_11.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure7_11.layout.width = '60%'
figure7_11.layout.height = '420px'
figure7_11.legend_location = 'bottom-left'
figure7_11.layout.left = '20px'

    
############################################################
############################################################
############################################################


#Plot data
############################################################
Title = 'Transition to Full Hiking Cycle' 
y_data = FullDataArray_Consec
y2_data = FullDataArray_Jumping
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '12px'},                 
                 label_location = 'end',
                 label = 'Price (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '0f',     
                 tick_style = {'font-size': '12px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'middle',
                 label = 'Count',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['Consecutive'],
                display_legend = True)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.4] * HistPlot.bins


#Histogram plot (price)
HistPlot2 = Hist(sample = y2_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['Jumping'],
                display_legend = True)    

#Setting histogram bar opacities 
HistPlot2.opacities = [0.4] * HistPlot2.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['', '.2f'], 
         labels = ['Count', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}

#Set up the tooltip interaction (price)
HistPlot2.tooltip = def_tt
HistPlot2.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure7_12 = Figure(marks = [HistPlot, HistPlot2], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure7_12.title = ( 'Figure 24: ' + Title +
                    ' Spreads Aggregated Data' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_12.title_style = {'font-size': '13px'}
figure7_12.background_style = {'fill': 'white'} 

figure7_12.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure7_12.layout.width = '45%'
figure7_12.layout.left = '60px'
figure7_12.layout.height = '450px'
figure7_12.legend_location = 'top-left'

In [49]:
VBox([widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ': </b>' + 
                   'Closer look at Consecutive and Quarterly (Jumping) Hike Spreads. ' +  
                   ' Frequency heatmap and day to maturity time series,' +
                   ' 180 days before expiry. </p>'),
      BorderHtmlW, VBox([HBox([figure7_8, figure7_9]), 
                         HBox([figure7_10, figure7_11])]), BorderHtmlW]) 

We can see on this stage that the similarity between consecutive and jumping spreads break, with Pc-bias getting priced in, consecutive spreads become rather obsolete as the quarterly-meeting leg becomes more and more relevant. Still, on this period the market was not completely sold on the idea that non-quarterly meetings were completely irrelevant, and would ocasionally skew them relative to their quarterly adjacent meetings once the frontmost meeting was discarded. 

All in all, these elements made these kind of spreads harder to trade relative to the previous stage, volatility shot upwards as hikes became imminent but the cycle was not yet underway, as the difference between quarterly and non-quarterly meetings grew, jumping spreads became the better alternative between the two. Here is a summary of observatios below:

<br>

|Consecutive Spreads|Jumping Spreads|
|-----------|-----------|
|PC-bias gets priced in, spreads become more directional <br> and trade almost always below 0|Spreads become more directional as hike spectations get <br> more polarized, 2016 spreads more even so than 2015 spreads|
|Larger trade ranges relative to the prior stage, <br> ~6bp to ~9bp range and an average daily std of ~0.42 basis points|Larger trade ranges relative to the prior stage, <br> ~7bp to ~9bp range and an average daily std of ~0.66 basis points|
|There is no clear pattern in terms of roll, some spreads displayed rolldown <br> while other moved higher coming close to expiration, as on this stage  <br> some non-quarterly meetings would get skewed as their <br> preceding quarterly meetings were discarded.|Most spreads displayed very strong rolldown moving  closer to expiration, <br> in spite of a flatter curve and higher front-end volatility, meetings would <br> quickly get discarded in favor of their subsequent alternatives <br> (quarterly and in come cases non-quarterly meetings as well).|

<br>
 
## Full-Hiking Cycle (2017-2018)

By this stage the Fed begins ramping up the tightening, hiking on every Quarterly meeting, excluding September 2017, where they announced the SOMA portfolio normalization. As a result, the curve inverts completely and the PC-bias becomes fully priced in. Non-Quarterly meetings are discarded pretty much completely and these spreads become virtually one-legged, as most of their price is set by the Quarterly meeting leg hike expectations. This is the worst scenario for these spreads, the "danger-zone", as there is no effective hedging, volatility is huge and we are basically trading naked Quarterly meetings.

In [50]:
#Title 
Title = 'Full Hiking Cycle'

#Set of hike spreads 
Spreads_set = ['G17H17', 'K17M17', 'N17U17', 
               'F18H18', 'K18M18', 'Q18U18']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
#Compute equally weighted portfolio of consecutive spreads
Portfolio_Consec = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                              len(GridFullData.columns)))
    
#Compute histogram (bins for all aggregated data)
FullDataArray_Consec = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))

FullBins = np.histogram(FullDataArray_Consec)

#Store bins for scatter
FullBins_Consec = FullBins


#Compute individual counts for each spread (same bins for all data)
CountDf =  pd.DataFrame(columns = Spreads_set)

for i in range(0, len(Spreads_set)):
    
    CountDf.iloc[:, i] = np.histogram(np.array(
                         GridFullData.iloc[:, i]), bins = FullBins[1])[0]
    
    
############################################################
############################################################
############################################################


#Plot data
############################################################
x_data = Spreads_set
y_data = FullBins[1]
MapData = np.array(CountDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
y_sc = LinearScale()
x_sc = OrdinalScale()
col_sc = ColorScale(min = float(np.min(MapData)), 
                    max = float(np.max(MapData)), 
    colors = ['#EDEDED', '#DE2369'])

ax_y = Axis(scale = x_sc, 
            label_location = 'end',
            label_offset = '40px',
            tick_style = {'font-size': '10px'}) 

ax_x = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label_location = 'middle',
            label = 'Price (bps)',
            label_offset = '40px', 
            num_ticks = len(y_data))

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Count', 
                   tick_format = '0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = y_data, 
                       column = x_data, 
                       color = MapData.T, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['column', 'color'], 
                 formats = ['', '0f'], 
                 labels = ['Hike Spread', 
                           'Count'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7_13 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y])

#Figure Title
figure7_13.title = ( 'Figure 19: ' + 
                   'Consecutive Spreads Frequency Map' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_13.title_style = {'font-size': '13px'}
figure7_13.background_style = {'fill': 'white'}

figure7_13.fig_margin = {'top':40, 'bottom':20, 'left':100, 'right':50}
figure7_13.layout.width = '35%'#'650px'#'35%'
figure7_13.layout.left = '10px'
figure7_13.layout.height = '395px'

    
############################################################
############################################################
############################################################


#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure7_14 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure7_14.title = ( 'Figure 20: ' +  
                   ' Consecutive Hike Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_14.title_style = {'font-size': '13px'}
figure7_14.background_style = {'fill': 'white'} 
    
figure7_14.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure7_14.layout.width = '60%'
figure7_14.layout.height = '420px'
figure7_14.legend_location = 'top-left'
figure7_14.layout.left = '20px'
figure7_14.legend_location = 'bottom-left'

In [51]:
#Set of hike spreads 
Spreads_set = ['H17M17', 'M17U17', 'U17Z17', 
               'H18M18', 'M18U18', 'U18Z18']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat

#Compute equally weighted portfolio of consecutive spreads
Portfolio_Jumping = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                               len(GridFullData.columns)))    
    
#Compute histogram (bins for all aggregated data)
FullDataArray_Jumping = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))

FullBins = np.histogram(FullDataArray_Jumping)


#Compute individual counts for each spread (same bins for all data)
CountDf =  pd.DataFrame(columns = Spreads_set)

for i in range(0, len(Spreads_set)):
    
    CountDf.iloc[:, i] = np.histogram(np.array(
                         GridFullData.iloc[:, i]), bins = FullBins[1])[0]  

    
############################################################
############################################################
############################################################


#Plot data
############################################################
x_data = Spreads_set
y_data = FullBins[1]
MapData = np.array(CountDf.T)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
y_sc = LinearScale()
x_sc = OrdinalScale()
col_sc = ColorScale(min = float(np.min(MapData)), 
                    max = float(np.max(MapData)), 
    colors = ['#EDEDED', '#01B0F0'])

ax_y = Axis(scale = x_sc, 
            label_location = 'end',
            label_offset = '40px',
            tick_style = {'font-size': '10px'}) 

ax_x = Axis(scale = y_sc, 
            orientation = 'vertical', 
            label_location = 'middle',
            label = 'Price (bps)',
            label_offset = '40px', 
            num_ticks = len(y_data))

col_ax = ColorAxis(scale = col_sc, 
                   label = 'Count', 
                   tick_format = '0f', 
                   label_offset = '50px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Hikes correlation plot (relevant hikes, PC/noPC)
grid_map = GridHeatMap(row = y_data, 
                       column = x_data, 
                       color = MapData.T, 
                       scales = {'column': x_sc, 
                                 'row': y_sc, 
                                 'color': col_sc}, 
                       opacity = 0.8, 
                       stroke = 'gray')
############################################################

#Step 3: Hover interaction
############################################################     
#Creating the tooltip
def_tt = Tooltip(fields = ['column', 'color'], 
                 formats = ['', '0f'], 
                 labels = ['Hike Spread', 
                           'Count'])

#Set up the tooltip interaction 
grid_map.tooltip = def_tt
grid_map.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################     
#initial Figure
figure7_15 = Figure(marks=[grid_map], 
                 axes = [ax_x, ax_y])

#Figure Title
figure7_15.title =  ( 'Figure 21: ' + 
                     'Jumping Spreads Frequency Map' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_15.title_style = {'font-size': '13px'}
figure7_15.background_style = {'fill': 'white'}

figure7_15.fig_margin = {'top':40, 'bottom':20, 'left':100, 'right':50}
figure7_15.layout.width = '35%'#'650px'#'35%'
figure7_15.layout.left = '10px'
figure7_15.layout.height = '395px'

    
############################################################
############################################################
############################################################


#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure7_16 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure7_16.title =  ( 'Figure 22: '  
                     ' Jumping Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure7_16.title_style = {'font-size': '13px'}
figure7_16.background_style = {'fill': 'white'}
    
figure7_16.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure7_16.layout.width = '60%'
figure7_16.layout.height = '420px'
figure7_16.legend_location = 'bottom-left'
figure7_16.layout.left = '20px'

In [52]:
VBox([widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ': </b>' + 
                   'Closer look at Consecutive and Quarterly (Jumping) Hike Spreads. ' +  
                   ' Frequency heatmap and day to maturity time series,' +
                   ' 180 days before expiry. </p>'),
      BorderHtmlW, VBox([HBox([figure7_13, figure7_14]), 
                         HBox([figure7_15, figure7_16])]), BorderHtmlW]) 

The trend that begun on the pre-hiking cycle continued on this stage, the distinction between consecutive and jumping spreads becomes very clear. The Pc-bias is fully priced by this stage, non-Quarterly meetings were consistently discarded as hikes were being priced almost exclusively on Quarterly meetings (no more skewing for the front-most non-Quarterly meeting as well). 

Trading consecutive spreads on this stage was pretty much pointless, they were extremely directional and more volatile than jumping spreads, as they were completely exposed to the Pc hikes curve level factor. Here is a summary of observatios below:

<br>

|Consecutive Spreads|Jumping Spreads|
|-----------|-----------|
|PC-bias was fully priced, non-Quarterly meetings were discarded. <br> Spreads were completely directional and traded always below 0|Spreads were directional, but in line with the shape of the curve. <br> This made them more "predictable", with a "stable" curve structure|
|Largest trade ranges relative to the prior stages, <br> ~12bp range and an average daily std of ~1.64 basis points|Similar trade ranges relative to the prior stage, <br> ~8bp to ~10bp range and an average daily std of ~0.77 basis points|
|Strong rolldown for most cases. The spreads that stayed close <br> to zero were exceptions: Sept17 (no hike and SOMA normalization), <br> and Mar17 (surprise hike for the market)|The curve was pretty stable on its structure, <br> that ment the roll was quite clear, particularly on 2018. <br> Spreads displayed strong rollup moving closer to expiration, <br> as the Fed continued aggressively hiking|

<br>

## Lets sum it all up

To sum up the previous sections, we take a look at the distribution for consecutive [`(Figure 23)`](#Figure_23) and jumping spreads [`(Figure 24)`](#Figure_23) aggregated data on 180 days before expiration, for the first two stages (2012 through 2016). We discard the third one as consecutive hike spread trading on that environment should be avoided: 

In [53]:
VBox([widgets.HTML('<a id="Figure_23"></a>'),
     widgets.HTML('<p style = "color: #125CB2;"> <b> Consecutive vs Jumping Spreads: </b>' + 
                   ' Pre Hiking Cycle and Transition to Full Hiking Cycle' + 
                   ' Aggregated Spread Data Histograms,' +
                   ' 180 days before expiry. </p>'),
      BorderHtmlW, HBox([figure7_6, figure7_12]), BorderHtmlW])

Some observations: 

>- The range for both set of spreads is way larger on the second stage, with more spread volatility, and direction. On the first stage both set of spreads were mean reverting around zero, which you would expect with a flatter curve.
>
>
>- On the first stage we can see that both set of spreads are similar, on the second stage we begin to see the pc-bias present, and both set of spreads become different type of strategies. 
>
>
>- For the second stage, we see a positive skewed distribution for consecutive spreads, as they tended to trade below zero with the quarterly-meeting leg becoming more relevant relative to the other leg. Jumping spreads, on the other hand were more balanced, although the distribution has a negative skew, due to hikes getting discarded quickly as the market priced the timing for the beginning of the cycle on the front-end.
>
>

<a id='Fourth'></a>
[<center> <h1 style = 'background-color:#D0C8D6; color:black; height:35px; padding: 5px;'> What happened with Consecutive Spreads Since The Christmas Rally?</h1> </center>](#TableOfContents)

## Overview

Fast forward to our current curve environment, we take a look at consecutive spreads and see what we can infer from that. We mentioned at the beginning of the document that developements outside the US, a change in tone on Fed Official's speeches and weak US economic data on Q4 2018 caused the market to abruptly change its view on the rate hike cycle and begin pricing a potential recession and rate cuts on the back-end.

The movement peaked around Christmas and the curve settled after that with a newish shape [`(Figure 9)`](#Figure_9), after having pretty much completely wiped out hike expectations. Our base case is that as long as a recession and a rate-cut cycle are priced on the back end, the curve could have a smooth enough slope to allow us to trade these consecutive spreads, in addition to jumping hike spreads, as rate cuts could be priced in a consecutive fashion.

We then look at what has happened with these spreads since the Christmas rally and try to draw some conclusions.

## Lets take a look at the data

We will be splitting consecutive spreads between 2019 and 2020 (F19H19 to Q19U19 and F20H20 to N20U20), as well as 2012-2014 spreads (aggregated data only, for the list of spreads go back to the third section). For simplicity's sake, we will refer to Non-Quarterly meetings as NoPc meetings, and Quarterly meetings as Pc meetings.


>- **How we will be looking at it:**  We are going to be using time series and aggregated spread data histograms.
>
>
>- **What are we looking for?:**  We are looking for differences between 2019 and 2020 spreads, if any. And to draw a line towards consecutive spreads on the Pre-Hiking Cycle stage we defined earlier, and see if there is any resemblance. 
>
>
>- **Samples:**  For current spreads, we are looking at data from January 2019 up to the most recent data available. For 2012-2014 spreads we will be looking at 180 trading days to maturity aggregated data. Because current spreads have smaller samples than 2012-2014 spreads, we are normalizing the histogram to avoid scaling issues.

In [54]:
#Set of hike spreads 
Spreads_set = ['F12H12', 'J12M12', 'Q12U12', 
               'F13H13', 'K13M13', 'N13U13', 
               'F14H14', 'J14M14', 'N14U14']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat

    
#Compute histogram (bins for all aggregated data)
FullDataArray_Consec = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))


############################################################
############################################################
############################################################


#Title 
Title = 'What Happened since The Christmas Rally?'


'''Spreads_set = ['F20H20', 'J20M20', 'N20U20', 
              'H20M20', 'M20U20', 'U20Z20']'''

#Set of hike spreads 
Spreads_set = ['K19M19', 'Q19U19',
               'F20H20', 'J20M20', 'N20U20']

#From date
FromDate = '2019/01'

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData.index = SpreadD[3][FromDate:].index


#Data arrays for histograms
FullDataArray_Current = np.reshape(np.array(GridFullData.T), 
                                   (1, len(GridFullData)*len(Spreads_set)))

FullDataArray_Current_2019 = np.reshape(np.array(GridFullData.iloc[:, 0:2].T), 
                                       (1, len(GridFullData)*len(Spreads_set[0:2])))

FullDataArray_Current_2020 = np.reshape(np.array(GridFullData.iloc[:, 2:].T), 
                                       (1, len(GridFullData)*len(Spreads_set[2:])))

#NET CHANGE
NetChangeSpreads = GridFullData.diff()


############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = FullDataArray_Current_2019
y2_data = FullDataArray_Current_2020

KDEData = KDEforPlot(FullDataArray_Current_2019[0])
x3_data = KDEData[0]
y3_data = KDEData[1]

KDEData = KDEforPlot(FullDataArray_Current_2020[0])
x4_data = KDEData[0]
y4_data = KDEData[1]

KDEData = KDEforPlot(FullDataArray_Consec[0])
x5_data = KDEData[0]
y5_data = KDEData[1]
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '11px'},                 
                 label_location = 'end',
                 label = 'Price (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '.2f',     
                 tick_style = {'font-size': '11px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'end',
                 label = 'Freq (normalized)',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 15,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['2019 Spreads'],
                display_legend = True)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.4] * HistPlot.bins


#Histogram plot (price)
HistPlot2 = Hist(sample = y2_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 15,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['2020 Spreads'],
                display_legend = True)    

#Setting histogram bar opacities 
HistPlot2.opacities = [0.4] * HistPlot2.bins


#Density plots
lin = Lines(x = x3_data, y = y3_data, 
            scales = {'x': Hist_x, 'y' : Hist_y},
            colors = ['#01B0F0'], 
            stroke_width = 3, interpolation = 'cardinal')

lin2 = Lines(x = x4_data, y = y4_data, 
            scales = {'x': Hist_x, 'y' : Hist_y},
            colors = ['#DE2369'], 
            stroke_width =2, interpolation = 'cardinal')

lin3 = Lines(x = x5_data, y = y5_data, 
            scales = {'x': Hist_x, 'y' : Hist_y},
            colors = ['black'], 
            stroke_width = 3, interpolation = 'cardinal', 
            labels = ['2012-2014 Spreads'], display_legend = True)
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['.2f', '.2f'], 
         labels = ['Freq', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}

#Set up the tooltip interaction (price)
HistPlot2.tooltip = def_tt
HistPlot2.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure21 = Figure(marks = [HistPlot, HistPlot2,
                           lin, lin2, lin3], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure21.title = ('Figure 25: ' +  
                  'Current Pc-NoPc Spreads Aggregated Data - ' +
                  'Base: ' + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure21.title_style = {'font-size': '13px'}
figure21.background_style = {'fill': 'white'} 

figure21.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure21.layout.width = '40%'
figure21.layout.left = '10px'
figure21.layout.height = '420px'
figure21.legend_location = 'top-left'


############################################################
############################################################
############################################################


#Plot data
############################################################ 
y_data = np.array(GridFullData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = GridFullData.index
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = DateScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Date',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure22 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure22.title =  ('Figure 26: ' +
                   ' Current Pc-NoPc Spreads Time Series - ' +
                   'Base: ' + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure22.title_style = {'font-size': '13px'}
figure22.background_style = {'fill': 'white'}
    
figure22.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure22.layout.width = '60%'
figure22.layout.height = '420px'
figure22.legend_location = 'top-left'

In [55]:
VBox([widgets.HTML('<a id="Figure_25"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
                   ' Current Consecutive Spread time series and aggregated Data Histograms,' +
                   ' split by year (2019 and 2020 Spreads) and including 2012-2014 spreads for comparison.' +
                   ' Histogram is normalized to avoid scaling issues (much more data for 2012-2014 spreads than for current ones).'
                   ' Settle data from ' + FromDate + ' to the most recent settle. </p>'),
      BorderHtmlW, HBox([figure21, figure22]), BorderHtmlW])

As soon as we look at current spreads time series since the Christmas rally [`(Figure 26)`](#Figure_25) we can clearly see there is an abrupt break for 2019 spreads by the end of March, triggered by a very dovish Fed (March FOMC meeting Statement and Economic projections, with 3 less hikes for 2019-2020 relative to the previous release) and weak Euro area PMI data. Both K19M19 and Q19U19 jumped by about 2 to 3 basis points in a market move prompted by a large drop on M19, U19 and Z19 as the market panicked and begun pricing te possibility of tactical cuts for 2019, without the need of clear signs of recession. 

2020 spreads, on the other hand, have shown resilience during the March event and a much more stable path, at least so far, similar to what we saw for consecutive spreads on the Pre-Hiking Cycle stage (2012-2014). This makes them more appealing as the slope of the curve on 2020 has remained smoother in spite of the March panic. We will be looking a bit closer at the March event later. 

If we look at their historical distributions [`(Figure 25)`](#Figure_25) we can confirm the same: 2019 spreads distribution is heavily skewed, with a huge right side tail due to spreads breaking after the March event. 2020 spreads have a more symetric distribution that has some resemblance to the 2012-2014 consecutive spreads distribution (black line), although much flatter, whether it moves closer to 2012-2014 or 2019 spreads will depend on the way the market considers Pc-bias as the curve rolls forward. 

## Summary

Considering what was mentioned above, we can then split, at least for now, 2019 and 2020 consecutive spreads and summarize as follows:

>- **2019 Spreads (F19H19 to Q19U19):** In spite of the Fed eliminating the Quarterly-meeting-only press conference pattern, the PC-bias remains priced in on 2019 to this day, but with a flatter curve and rate cuts. The curve resembles the transition sub-period of H2 2014 through 2016, that is, a flat curve with PC-bias and positive/flat slope. These spreads are rather unpredictable, quite volatile relative to their 2020 peers, and still very exposed to the Quarterly meeting leg.  
>
>
>- **2020 Spreads (F20H20 to N20U20):** We have observed so far that the market is persistently pricing a smoother positve sloped curve for 2020 with fading PC-bias, likely related to recession and emergency (consecutive) rate cuts expectations. This section of the curve resembles the Pre-hiking cycle scenario, and so far spreads have behaved very similarly to their 2012-2014 counterparts. It is still early to tell and PC-bias could easily be back on the curve under several potential scenarios (tactical cuts, back to rate hikes, mainly). 

<a id='Fifth'></a>
[<center> <h1 style = 'background-color:#D0C8D6; color:black; height:35px; padding: 5px;'> Useful Questions and (Some) Answers </h1> </center>](#TableOfContents)


## What Happens if a Non-Quarterly Meeting Gets Skewed?


### Some background

During the Transition to a full Hiking Cycle stage (2015 through 2016), in spite of the Press Conference Bias being fully priced in by the market, as the cycle had not began fully, some Non-Quarterly meetings from April 2015 onward got skewed relative to their adjacent Quarterly meetings.The full list included: Apr15, Jul15, Oct15, Apr16 and Jul16. 

Back then the market was expecting the beginning of the cycle and an imminent rate hike that could come any meeting. In spite of the PC-bias being taken into consideration, whenever the market expected a rate hike on the front-end, it would skew the first Non-Quarterly meeting giving it higher rate hike expectations once the previous Quarterly one began showing sings of being potentially discarded by the Fed. Whether it was due to economic data, Fedspeak or other drivers, the process repeated several times. This was an intuitive process by the market, as market participants had no prior evidence of the Fed actually choosing Quarterly meetings for rate moves yet. 

### How is this relevant for the current curve environment?

Remember we stated at the beginning of this document that we were focusing our analysis on spreads where we could short the non-quarterly meeting vs the adjacent quarterly meeting, but only when the Non-Quarterly meeting was the first leg of the spread? 

In spite of us being on a different environment right now (potential cuts, and PC-bias confirmation by the Fed), cuts are being priced on every consecutive meeting, which means a smoother curve and the potential for Non-Quarterly meetings revisiting the same behavior we saw on the Trasition stage: getting skewed on the front-end as the frontmost Quarterly meeting gets discarded for a potential cut. 

It is for this reason we believe it to be much safer to trade spreads that meet the criteria stated above, avoiding the risk of the Quarterly leg getting completely discarded.


### Lets take a look at the data

We can split consecutive spreads that involved the particular Non-Quarterly meetings that got skewed ( Apr15, Jul15, Oct15, Apr16 and Jul16), during the transition period in two groups, based on where the Non-Quarterly meeting is placed (as the front or back leg of the spread), and compare them. For simplicity's sake, we will refer to Non-Quarterly meetings as NoPc meetings, and Quarterly meetings as Pc meetings.

We will use the same type of plots we described on the previous section, fixed day to maturity time series, aggregated spread data histograms to compare both type of spreads and look at their distributions, and aggregated spread data fixed day to maturity time series:

>- **Fixed Day to Maturity Time Series:** The idea behind using these plots is to take a closer look at how each spread behaved 180 trading days before their expiration and see if there is any pattern (Rollup or Rolldown) as they approach expiration and most importantly, any spikes that indicate the Non-Quarterly leg getting biased.
> 
>
>- **Aggregated Data Histograms:** We aggregate all of our 180 days to maturity data (all of the spreads) and plot their distribution, we do this for both sets of spreads separatedly. The idea is to see if there is any difference between them, we want to look at their distribution's shapes, their tails, see if they are centered around different means, etc. 
>
>
>- **Aggregated Data Day to Maturity Time Series:** This is simply an aggregation of individual spreads day to maturity time series, we do it by creating an equally weighted "portfolio" of all Quarterly-NonQuarterly meeting spreads and repeat the same process for NonQuarterly-Quarterly meeting spreads, its a summary of what we saw individually for each spread on their corresponding day to maturity time series. 

In [56]:
#Title 
Title = 'What Happens if NoPc Meetings Get Skewed?'

#Set of hike spreads 
Spreads_set = ['J15M15', 'N15U15', 'V15Z15', 
               'J16M16', 'N16U16']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
#Compute equally weighted portfolio of consecutive spreads
Portfolio_NoPC_PC = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                              len(GridFullData.columns)))

#Data array for histogram
FullDataArray_NoPC_PC = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))


############################################################
############################################################
############################################################


#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True) 
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure17 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure17.title =  ('Figure 27: ' + 
                   ' - NoPc-Pc Hike Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure17.title_style = {'font-size': '13px'}
figure17.background_style = {'fill': 'white'}
    
figure17.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure17.layout.width = '50%'
figure17.layout.height = '420px'

figure17.legend_location = 'top-left'


############################################################
############################################################
############################################################


#Set of hike spreads 
Spreads_set = ['H15J15', 'M15N15', 'U15V15', 
               'H16J16', 'M16N16']

#Number of days
NDays = 180

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
#Compute equally weighted portfolio of consecutive spreads
Portfolio_PC_NoPC = GridFullData.dot(np.repeat(1/len(GridFullData.columns), 
                                               len(GridFullData.columns)))

#Data array for histogram
FullDataArray_PC_NoPC = np.reshape(np.array(GridFullData.T), 
                                   (1, NDays*len(Spreads_set)))


############################################################
############################################################
############################################################


#Plot data
############################################################ 
DtMatData = GetDtMatTsData(Spreads_set, 
                           Data = FFsDataList, TsDtM = NDays, 
                           Hikes = True)

y_data = np.array(DtMatData).transpose().astype('float')

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = Spreads_set, 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC', '#31DC8D',
                                    '#DE2369', '#F53F52',
                                    '#F05471'], 
               stroke_width = 1, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure18 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure18.title =  ('Figure 28: ' 
                   ' - Pc-NoPc Hike Spreads DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure18.title_style = {'font-size': '13px'}
figure18.background_style = {'fill': 'white'}
    
figure18.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure18.layout.width = '50%'
figure18.layout.height = '420px'
figure18.legend_location = 'bottom-left'

In [57]:
VBox([widgets.HTML('<a id="Figure_27"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' NoPc-Pc and Pc-NoPc Consecutive Hike Spreads that experienced' + 
             ' a skewing of their NoPc leg.'      
             ' Day to maturity time series,' +
             ' 180 days before expiry. </p>'),
      BorderHtmlW, HBox([figure17, figure18]), BorderHtmlW])

The goal here is to see how the skewing of the Non-Quarterly meeting affected each type of spread. Our base case is that the skewing should be more apparent on those spreads that have the Non-Quarterly meeting as the back leg. The intuition behind this is that the skewing occurs once the frontmost Quarterly meeting is getting discarded, as it approaches expiration, in favor of the meetings that proceed it.

Looking at both [`Figure 27`](#Figure_27) and [`Figure 28`](#Figure_27), we can see the following: 

>- The skewing on most spreads occurred around the 100-90 days to maturity mark, this is more apparent on the Pc-NoPc series. 
>
>
>- Most Pc-NoPc spreads had a way more aggressive reaction once the frontmost Pc leg got discarded and the NoPc adjacent meeting picked up some of its hike pricing, moving on some cases more than 5 basis points over the course of 3 months.
>
>
>- Most NoPc-Pc traded on a more stable range even after the Non-Quarterly leg got skewed on the front end (save for N16U16). 
>
>

In [58]:
#Plot data
############################################################
y_data = FullDataArray_NoPC_PC
y2_data = FullDataArray_PC_NoPC
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '11px'},                 
                 label_location = 'end',
                 label = 'Price (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '0f',     
                 tick_style = {'font-size': '11px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'end',
                 label = 'Count',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['NoPc-Pc Spreads'],
                display_legend = True)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.4] * HistPlot.bins


#Histogram plot (price)
HistPlot2 = Hist(sample = y2_data, 
                normalized =  False,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 20,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['Pc-NoPc Spreads'],
                display_legend = True)    

#Setting histogram bar opacities 
HistPlot2.opacities = [0.4] * HistPlot2.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['', '.2f'], 
         labels = ['Count', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}

#Set up the tooltip interaction (price)
HistPlot2.tooltip = def_tt
HistPlot2.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure19 = Figure(marks = [HistPlot, HistPlot2], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure19.title = ('Figure 29: ' + 
                  'Consecutive Hike Spreads Aggregated Data' + 
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure19.title_style = {'font-size': '13px'}
figure19.background_style = {'fill': 'white'} 

figure19.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure19.layout.width = '40%'
figure19.layout.left = '10px'
figure19.layout.height = '420px'
figure19.legend_location = 'top-right'


############################################################ 
############################################################ 
############################################################ 


#Plot data
############################################################ 
y_data = np.array([np.array(Portfolio_NoPC_PC), 
                   np.array(Portfolio_PC_NoPC)])

#Indices (X tick labels, days to maturity)
x_data = DtMatData.index.days
x_data = [str(DtMatData.index.days[i]) + 
                ' days'  for i in range(0, len(x_data))] 
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = OrdinalScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Days to Maturity',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Price (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['NoPc-Pc Spreads', 
                         'Pc-NoPc Spreads'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#01B0F0', '#DE2369'], 
               stroke_width = 2,
               marker = 'circle', marker_size = 15, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['', '.2f'], 
         labels = ['DTM', 'Price'])

TsPlot.tooltip = def_tt
TsPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure20 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure20.title = ('Figure 30: ' +  
                  ' Consecutive Hike Spreads Aggregated Data DTM Series' +
                   ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure20.title_style = {'font-size': '13px'}
figure20.background_style = {'fill': 'white'}
    
figure20.fig_margin = {'top':40, 'bottom':50, 'left':80, 'right':20}
figure20.layout.width = '60%'
figure20.layout.height = '420px'
figure20.legend_location = 'top-right'

In [59]:
VBox([widgets.HTML('<a id="Figure_29"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' NoPc-Pc and Pc-NoPc Consecutive Hike Spreads that experienced' + 
             ' a skewing of their NoPc leg.'      
             ' Aggregated data histogram and day to maturity time series,' +
             ' 180 days before expiry.' + 
             ' Data is aggregated by computing an equally weighted portfolio of all spreads. </p>'),
      BorderHtmlW, HBox([figure19, figure20]), BorderHtmlW])

If we look at both set of spreads aggregated data, we can confirm our base case:  

>- Looking at their aggregated data distribution [`(figure 29)`](#Figure_29), we can see that these are two different type of trades. They do involve the same type of meetings, but the Pc-NoPc spreads have a fatter left side tail compared to the right side tail of the NoPc-Pc set. The skewing of the Non-Quarterly leg being more apparent on the first set of spreads, due to the discarding of the frontmost Quarterly leg. This does not appear to be as frequent on the NoPc-Pc spreads, we can see that most of the data for this set remains below zero.  
>
>
>- If we look at aggregated DTM time series for both sets [`(figure 30)`](#Figure_29), we can also see clearly how around the 90 days to maturity mark, Pc-NoPc spreads begin to move towards negative as the frontmost Quarterly meeting gets discarded in favor of the Non-Quarterly adjacent meeting. The skewing is not so apparent on NoPc-Pc spreads, they do move but not quite as the first set, and remain on average below zero (Quarterly leg > Non-Quarterly leg). 


## How About Leg Behaviour, Are We Just Trading PC Level?

This question is important if we consider our base case, since for these spreads to work, we need the curve to have a smooth slope without seasonal Pc-bias type of effects priced in. As reviewed above, we come from the Full Rate Hike Cycle that spanned from 2017 through 2018, in which most of the rate hike expectations were traded around quarterly meetings. The Press Conference Meeting Bias was strong then and there is still a remnant of it priced in by the market, specially on the front-end of the curve. Thus the question that comes up: 

Are moves on current consecutive spreads, for the most part, a result of the Quarterly-meeting leg volatility? are we trading with a fragile hedge then, completely exposed to the Quarterly-meeting leg? 

### Short Answer

The short answer is...yes. If we aggregate Consecutive Spreads data since the Christmas rally, without separating 2019 from 2020 spreads and look at the daily settle-to-settle net changes for both their legs [`(Figure 31)`](#Figure_31), we can see that, on average, the PC leg moved on a wider range, with the NoPc leg not only moving on a smaller range but also displaying higher frequency on the same set of prices: its distribution has smaller tails and higher kurtosis (way more concentrated around zero) than the PC leg net change distribution. The pattern becomes a bit clearer if we look at the scatterplot of daily net changes for both legs [`(Figure 32)`](#Figure_31). 

In [60]:
#Title
Title = 'Are We Just Trading PC Level?'

##NO PC LEG
legs = ['K19', 'Q19', 'F20', 'J20', 'N20']

#From date
FromDate = '2019/01'

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangeNoPcLegs = GridFullData_legs.diff()

#Compute histogram (bins for all aggregated data)
FullDataArray_NcNoPc = np.reshape(np.array(NetChangeNoPcLegs.T), 
                                   (1, len(NetChangeNoPcLegs)*len(legs)))


##PC LEG
legs = ['M19', 'U19', 'H20', 'M20', 'U20']

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangePcLegs = GridFullData_legs.diff()

#Compute histogram (bins for all aggregated data)
FullDataArray_NcPc = np.reshape(np.array(NetChangePcLegs.T), 
                                   (1, len(NetChangePcLegs)*len(legs)))


############################################################ 
############################################################ 
############################################################ 


#Set of hike spreads 
Spreads_set = ['F12H12', 'J12M12', 'Q12U12', 
               'F13H13', 'K13M13', 'N13U13', 
               'F14H14', 'J14M14', 'N14U14']

##NO PC LEG
legs = ['F12', 'J12', 'Q12', 
        'F13', 'K13', 'N13',
        'F14', 'J14', 'N14']

#Number of days
NDays = 180

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat

    
#Net change
NetChangeNoPcLegs_2 = GridFullData_legs.diff()

#Compute histogram (bins for all aggregated data)
FullDataArray_NcNoPc_2 = np.reshape(np.array(NetChangeNoPcLegs_2.T), 
                                   (1, len(NetChangeNoPcLegs_2)*len(legs)))

##PC LEG
legs = ['H12', 'M12', 'U12', 
        'H13', 'M13', 'U13', 
        'H14', 'M14', 'U14']

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    Strategy = Spreads_set[i]

    TimeSpan = GetSpreadDates(Strategy, FFsDataList)
    Date_1 = TimeSpan[1][0]
    Date_2 = TimeSpan[1][1]

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][Date_1:Date_2]).T[0][-NDays:]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat 

#Net change
NetChangePcLegs_2 = GridFullData_legs.diff()

#Compute histogram (bins for all aggregated data)
FullDataArray_NcPc_2 = np.reshape(np.array(NetChangePcLegs_2.T), 
                                   (1, len(NetChangePcLegs_2)*len(legs)))


############################################################ 
############################################################ 
############################################################ 


#Plot data
############################################################
y_data = FullDataArray_NcNoPc
y2_data = FullDataArray_NcPc
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '12px'},                 
                 label_location = 'end',
                 label = 'Net Change (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '.2f',     
                 tick_style = {'font-size': '12px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'end',
                 label = 'Freq (normalized)',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 10,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['NoPc Leg NC'],
                display_legend = True)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.4] * HistPlot.bins


#Histogram plot (price)
HistPlot2 = Hist(sample = y2_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 10,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['Pc Leg NC'],
                display_legend = True)    

#Setting histogram bar opacities 
HistPlot2.opacities = [0.4] * HistPlot2.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['.2f', '.2f'], 
         labels = ['Freq', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}

#Set up the tooltip interaction (price)
HistPlot2.tooltip = def_tt
HistPlot2.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure24 = Figure(marks = [HistPlot, HistPlot2], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure24.title = ('Figure 31: ' +  
                  ' - Consecutive Spreads Aggregated Leg NC Data - ' + 
                  'Base: ' + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure24.title_style = {'font-size': '13px'}
figure24.background_style = {'fill': 'white'} 

figure24.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure24.layout.width = '45%'
figure24.layout.left = '30px'
figure24.layout.height = '450px'
figure24.legend_location = 'top-left'


############################################################ 
############################################################ 
############################################################ 


#Plot data
############################################################
y_data = FullDataArray_NcNoPc[0][1:]
y2_data = FullDataArray_NcPc[0][1:]
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '12px'},                 
                 label_location = 'end',
                 grid_color = '#EDEDED',
                 label = 'NoPcLeg Nc (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '.2f',     
                 tick_style = {'font-size': '12px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'end',
                 label = 'PcLeg Nc (bps)',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################


ScattPlot = Scatter(x = y_data, 
                     y = y2_data,
                scales = {'x': Hist_x, 
                          'y': Hist_y}, 
                     display_names = False, 
                     colors = ['#DE2369'],
                     default_size = 70, 
                     unhovered_style={'opacity': 0.1}) 
############################################################    

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['.2f', '.2f'], 
                 labels = ['NoPcLeg Nc (bps)', 
                           'PcLeg Nc (bps)'])

#Set the tooltip interaction(s)
ScattPlot.tooltip = def_tt
ScattPlot.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure24_2 = Figure(marks = [ScattPlot], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure24_2.title = ('Figure 32: ' 
                  ' - Consecutive Spreads Aggregated Leg NC Data - ' +
                  'Base: ' + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure24_2.title_style = {'font-size': '13px'}
figure24_2.background_style = {'fill': 'white'} 

figure24_2.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure24_2.layout.width = '45%'
figure24_2.layout.left = '60px'
figure24_2.layout.height = '450px'
figure24_2.legend_location = 'top-left'


############################################################ 
############################################################ 
############################################################ 


#Plot data
############################################################
y_data = FullDataArray_NcNoPc_2
y2_data = FullDataArray_NcPc_2
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '12px'},                 
                 label_location = 'end',
                 label = 'Net Change (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '.2f',     
                 tick_style = {'font-size': '12px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'end',
                 label = 'Freq (normalized)',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 10,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['NoPc Leg NC'],
                display_legend = True)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.4] * HistPlot.bins


#Histogram plot (price)
HistPlot2 = Hist(sample = y2_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 10,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['Pc Leg NC'],
                display_legend = True)    

#Setting histogram bar opacities 
HistPlot2.opacities = [0.4] * HistPlot2.bins
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['.2f', '.2f'], 
         labels = ['Freq', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}

#Set up the tooltip interaction (price)
HistPlot2.tooltip = def_tt
HistPlot2.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure25 = Figure(marks = [HistPlot, HistPlot2], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure25.title = ('Figure 33: ' +
                  ' - Consecutive Spreads Aggregated Leg NC Data' +
                  ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure25.title_style = {'font-size': '13px'}
figure25.background_style = {'fill': 'white'} 

figure25.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure25.layout.width = '45%'
figure25.layout.left = '30px'
figure25.layout.height = '450px'
figure25.legend_location = 'top-left'


############################################################ 
############################################################ 
############################################################ 


#Plot data
############################################################
y_data = FullDataArray_NcNoPc_2[0][1:]
y2_data = FullDataArray_NcPc_2[0][1:]
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '12px'},                 
                 label_location = 'end',
                 grid_color = '#EDEDED',
                 label = 'NoPcLeg Nc (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '.2f',     
                 tick_style = {'font-size': '12px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'end',
                 label = 'PcLeg Nc (bps)',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
ScattPlot = Scatter(x = y_data, 
                     y = y2_data,
                scales = {'x': Hist_x, 
                          'y': Hist_y}, 
                     display_names = False, 
                     colors = ['#DE2369'],
                     default_size = 70,  
                     unhovered_style={'opacity': 0.1}) 
############################################################  

#Step 3: Hover interaction
############################################################       
#Tooltip (main scatter)
def_tt = Tooltip(fields = ['x', 'y'], 
                 formats = ['.2f', '.2f'], 
                 labels = ['NoPcLeg Nc (bps)', 
                           'PcLeg Nc (bps)'])

#Set the tooltip interaction(s)
ScattPlot.tooltip = def_tt
ScattPlot.interactions = {'hover': 'tooltip'}
############################################################ 

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure25_2 = Figure(marks = [ScattPlot], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure25_2.title = ('Figure 34: ' +
                  ' - Consecutive Spreads Aggregated Leg NC Data' +
                  ' - ' + str(NDays) + ' DTM')

#Figure layout (except size attributes, defined on section 4)
figure25_2.title_style = {'font-size': '13px'}
figure25_2.background_style = {'fill': 'white'} 

figure25_2.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure25_2.layout.width = '45%'
figure25_2.layout.left = '60px'
figure25_2.layout.height = '450px'
figure25_2.legend_location = 'top-left'

In [61]:
VBox([widgets.HTML('<a id="Figure_31"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' Current Consecutive Hike Spreads Leg Net Change Data.' + 
             ' Aggregated data histogram and Scatterplot (both legs).' +
             ' Settle data from ' + FromDate + ' to the most recent settle.' + 
             ' Data is aggregated by computing an equally weighted portfolio of all spreads. </p>'),
      BorderHtmlW, HBox([figure24, figure24_2]), BorderHtmlW])

Here are the same set of plots, this time for the Pre-Hiking Cycle spreads (2012-2014). Here that the pattern mentioned above is not so clear [`(Figure 34)`](#Figure_33), the PC leg does have a bit more dispersion than the NoPc leg, its tails are fatter and its data is less concentrated [`(Figure 33)`](#Figure_33) but the NoPc leg is not as stable as it is on current spreads, and it resembles the other leg much better, making it a better hedge. 

In [62]:
VBox([widgets.HTML('<a id="Figure_33"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' 2012-2014 Consecutive Hike Spreads Leg Net Change Data.' + 
             ' Aggregated data histogram and Scatterplot (both legs),' +
             ' 180 days before expiry.' + 
             ' Data is aggregated by computing an equally weighted portfolio of all spreads. </p>'),
      BorderHtmlW, HBox([figure25, figure25_2]), BorderHtmlW])

In [63]:
#Set of hike spreads 
Spreads_set = ['K19M19', 'Q19U19', 
               'F20H20', 'J20M20', 'N20U20']

#From date
FromDate = '2019/01'

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData.index = SpreadD[3][FromDate:].index

#NET CHANGE
NetChangeSpreads = GridFullData.diff()


##PC LEG
legs = ['M19', 'U19', 'H20', 'M20', 'U20'] 

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangePcLegs = GridFullData_legs.diff()


##NO PC LEG
legs = ['K19', 'Q19', 'F20', 'J20', 'N20']


#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangeNoPcLegs = GridFullData_legs.diff()


############################################################
############################################################
############################################################


Portfolio_Spreads = (NetChangeSpreads * -2).dot(np.repeat(1/len(NetChangeSpreads.columns), 
                     len(NetChangeSpreads.columns)))

Portfolio_PcLegs = NetChangePcLegs.dot(np.repeat(1/len(NetChangePcLegs.columns), 
                     len(NetChangePcLegs.columns)))

Portfolio_NoPcLegs = NetChangeNoPcLegs.dot(np.repeat(1/len(NetChangeNoPcLegs.columns), 
                     len(NetChangeNoPcLegs.columns)))

Portfolio_Spreads = abs(Portfolio_Spreads)

bins = 5
SpreadsHist, Spreads_edges = np.histogram(Portfolio_Spreads[~np.isnan(Portfolio_Spreads)], bins = bins)

#VARIANCE
#Compute variance of the average PC and no PC legs, as per the cumulative distribution of the 
#average noPc-Pc spreads portfolio
VarianceOutDf = pd.DataFrame([], index = ['Spreads Quantiles', 'PcLeg Variance', 
                               'NoPcLeg Variance', 'Total Variance', 
                               'PcLeg share Total Variance', 
                               'NoPcLeg share Total Variance'], 
                             columns = ['Q' + str(k) for k in np.arange(1/bins, 1.1, 1/bins)])

#Fill the data frame
for i in range(1, len(Spreads_edges)):
    
    ithEdgeIndices = Portfolio_Spreads[Portfolio_Spreads <= Spreads_edges[i]].index
    
    #Variances
    ithEdge_Pc_var = Portfolio_PcLegs[ithEdgeIndices].var()
    ithEdge_NoPc_var = Portfolio_NoPcLegs[ithEdgeIndices].var()
    
    Total_var = ithEdge_Pc_var + ithEdge_NoPc_var 
    PcShare_var = ithEdge_Pc_var / Total_var
    NoPcShare_var = ithEdge_NoPc_var / Total_var    
    
    VarianceOutDf.iloc[0, i - 1] = Spreads_edges[i]
    VarianceOutDf.iloc[1, i - 1] = ithEdge_Pc_var
    VarianceOutDf.iloc[2, i - 1] = ithEdge_NoPc_var
    VarianceOutDf.iloc[3, i - 1] = Total_var
    VarianceOutDf.iloc[4, i - 1] = PcShare_var
    VarianceOutDf.iloc[5, i - 1] = NoPcShare_var


############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = np.array(VarianceOutDf.iloc[4, :])
y2_data = np.array([np.array(VarianceOutDf.iloc[4, :]), 
                    np.repeat(1, bins)])
y3_data = np.array([np.array(VarianceOutDf.iloc[4, :]), 
                    np.repeat(0, bins)])

#Indices (X tick labels, days to maturity)
x_data = np.arange(1/bins, 1.1, 1/bins)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_sc = LinearScale() 
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_sc, 
                   tick_style = {'font-size': '12px'}, 
                   tick_format = '0.2f',
                   label_location = 'end',
                   label = 'Spreads Nc Cumulative Distribution',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.2f', 
                   tick_style = {'font-size': '12px'},
                   label_location = 'end',
                   label = 'Share of Total Variance',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = [], 
               scales = {'x': MainTs_x_sc, 
                         'y' : MainTs_y_sc},
               colors = ['black'], #, '#31DC8D'
               stroke_width = 4,
               marker = 'circle',
               interpolation = 'cardinal',
               marker_size = 120,
               visible = True)

TsPlot2 = Lines(x = x_data, 
               y = y2_data, 
               scales = {'x': MainTs_x_sc, 
                         'y' : MainTs_y_sc},
               stroke_width = 0,
               interpolation = 'cardinal',
               fill = 'between',  
               fill_colors = ['#01B0F0'],  
               fill_opacities = ['0.85'],
               visible = True)

TsPlot3 = Lines(x = x_data, 
               y = y3_data, 
               scales = {'x': MainTs_x_sc, 
                         'y' : MainTs_y_sc},
               stroke_width = 0,
               interpolation = 'cardinal',
               fill = 'between',  
               fill_colors = ['#DE2369'],  #'#DE2369' 
               fill_opacities = ['0.85'],
               visible = True)

LabelsPlot = Label(x = [0.03, 0.03], y = [0.90, 0.06], 
                   text = ['NoPcLeg Share of Total Variance', 
                           'PcLeg Share of Total Variance'], 
                   default_size = 18, font_weight = 'bolder', 
                   colors = ['white', 'white'], 
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['.2f', '.2f'], 
         labels = ['Spread Nc Quantile', 
                   'PcLeg Share of Variance'])

TsPlot.tooltip = def_tt
TsPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure32 = Figure(marks = [TsPlot2, TsPlot3, TsPlot, LabelsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure32.title =  ('Figure 35: ' 
                  ' Current Spreads Leg Share of NC Total Variance - ' + 
                   'Base: ' + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure32.title_style = {'font-size': '13px'}
figure32.background_style = {'fill': 'white'}
    
figure32.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure32.layout.width = '45%'
figure32.layout.height = '450px'
figure32.legend_location = 'bottom-left'
figure32.layout.left = '30px'


############################################################
############################################################
############################################################


#Set of hike spreads 
Spreads_set = ['K13M13', 'N13U13', 
               'F14H14', 'J14M14', 'N14U14']

#From&to date
FromDate = '2012/01'
ToDate = '2013/04'

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:ToDate]).T[0]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData.index = SpreadD[3][FromDate:ToDate].index

#NET CHANGE
NetChangeSpreads_2 = GridFullData.diff()


##NO PC LEG
legs = ['K13', 'N13',
        'F14', 'J14', 'N14']

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:ToDate]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:ToDate].index  
    

#NET CHANGE
NetChangePcLegs_2 = GridFullData_legs.diff()

##PC LEG
legs = ['M13', 'U13', 
        'H14', 'M14', 'U14']

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:ToDate]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:ToDate].index  
    

#NET CHANGE
NetChangeNoPcLegs_2 = GridFullData_legs.diff()


############################################################
############################################################
############################################################


Portfolio_Spreads_2 = (NetChangeSpreads_2 * -2).dot(np.repeat(1/len(NetChangeSpreads_2.columns), 
                     len(NetChangeSpreads_2.columns)))

Portfolio_PcLegs_2 = NetChangePcLegs_2.dot(np.repeat(1/len(NetChangePcLegs_2.columns), 
                     len(NetChangePcLegs_2.columns)))

Portfolio_NoPcLegs_2 = NetChangeNoPcLegs_2.dot(np.repeat(1/len(NetChangeNoPcLegs_2.columns), 
                     len(NetChangeNoPcLegs_2.columns)))

Portfolio_Spreads_2 = abs(Portfolio_Spreads_2)

bins = 5
SpreadsHist_2, Spreads_edges_2 = np.histogram(Portfolio_Spreads_2[~np.isnan(Portfolio_Spreads_2)], bins = bins)

#VARIANCE
#Compute variance of the average PC and no PC legs, as per the cumulative distribution of the 
#average noPc-Pc spreads portfolio
VarianceOutDf_2 = pd.DataFrame([], index = ['Spreads Quantiles', 'PcLeg Variance', 
                               'NoPcLeg Variance', 'Total Variance', 
                               'PcLeg share Total Variance', 
                               'NoPcLeg share Total Variance'], 
                             columns = ['Q' + str(k) for k in np.arange(1/bins, 1.1, 1/bins)])

#Fill the data frame
for i in range(1, len(Spreads_edges_2)):
    
    ithEdgeIndices = Portfolio_Spreads_2[Portfolio_Spreads_2 <= Spreads_edges_2[i]].index
    
    #Variances
    ithEdge_Pc_var = Portfolio_PcLegs_2[ithEdgeIndices].var()
    ithEdge_NoPc_var = Portfolio_NoPcLegs_2[ithEdgeIndices].var()
    
    Total_var = ithEdge_Pc_var + ithEdge_NoPc_var 
    PcShare_var = ithEdge_Pc_var / Total_var
    NoPcShare_var = ithEdge_NoPc_var / Total_var    
    
    VarianceOutDf_2.iloc[0, i - 1] = Spreads_edges_2[i]
    VarianceOutDf_2.iloc[1, i - 1] = ithEdge_Pc_var
    VarianceOutDf_2.iloc[2, i - 1] = ithEdge_NoPc_var
    VarianceOutDf_2.iloc[3, i - 1] = Total_var
    VarianceOutDf_2.iloc[4, i - 1] = PcShare_var
    VarianceOutDf_2.iloc[5, i - 1] = NoPcShare_var


############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = np.array(VarianceOutDf_2.iloc[4, :])
y2_data = np.array([np.array(VarianceOutDf_2.iloc[4, :]), 
                    np.repeat(1, bins)])
y3_data = np.array([np.array(VarianceOutDf_2.iloc[4, :]), 
                    np.repeat(0, bins)])

#Indices (X tick labels, days to maturity)
x_data = np.arange(1/bins, 1.1, 1/bins)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_sc = LinearScale() 
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_sc, 
                   tick_style = {'font-size': '12px'}, 
                   tick_format = '0.2f',
                   label_location = 'end',
                   label = 'Spreads Nc Cumulative Distribution',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.2f', 
                   tick_style = {'font-size': '12px'},
                   label_location = 'end',
                   label = 'Share of Total Variance',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = [], 
               scales = {'x': MainTs_x_sc, 
                         'y' : MainTs_y_sc},
               colors = ['black'], #, '#31DC8D'
               stroke_width = 4,
               marker = 'circle',
               interpolation = 'cardinal',
               marker_size = 120,
               visible = True)

TsPlot2 = Lines(x = x_data, 
               y = y2_data, 
               scales = {'x': MainTs_x_sc, 
                         'y' : MainTs_y_sc},
               stroke_width = 0,
               interpolation = 'cardinal',
               fill = 'between',  
               fill_colors = ['#01B0F0'],  
               fill_opacities = ['0.85'],
               visible = True)

TsPlot3 = Lines(x = x_data, 
               y = y3_data, 
               scales = {'x': MainTs_x_sc, 
                         'y' : MainTs_y_sc},
               stroke_width = 0,
               interpolation = 'cardinal',
               fill = 'between',  
               fill_colors = ['#DE2369'],  #'#DE2369' 
               fill_opacities = ['0.85'],
               visible = True)

LabelsPlot = Label(x = [0.03, 0.03], y = [0.90, 0.06], 
                   text = ['NoPcLeg Share of Total Variance', 
                           'PcLeg Share of Total Variance'], 
                   default_size = 18, font_weight = 'bolder', 
                   colors = ['white', 'white'], 
                   enable_move = True) 
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['.2f', '.2f'], 
         labels = ['Spread Nc Quantile', 
                   'PcLeg Share of Variance'])

TsPlot.tooltip = def_tt
TsPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure33 = Figure(marks = [TsPlot2, TsPlot3, TsPlot, LabelsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure33.title =  ('Figure 36: ' 
                  ' 2013/2014 Spreads Leg Share of NC Total Variance - ' + 
                   'Base: ' + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure33.title_style = {'font-size': '13px'}
figure33.background_style = {'fill': 'white'}
    
figure33.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure33.layout.width = '45%'
figure33.layout.height = '450px'
figure33.legend_location = 'bottom-left'
figure33.layout.left = '60px'

To help us settle the matter, lets look at [`Figures 35 and 36`](#Figure_35) below (for current spreads for Pre-Hiking Cycle spreads). These plots allow us to see how the share of total leg net-change variance corresponding to the Pc-leg evolves as daily net changes for consecutive spreads grow larger (we use absolute values for daily net changes). In another words, this is a way of seeing wether the PC-leg volatility is behind spread moves as they grow larger on either direction, on a daily settle-to-settle basis:

In [64]:
VBox([widgets.HTML('<a id="Figure_35"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' Current and 2013-2014 Consecutive Hike Spreads Aggregated Leg Net Change Data.' + 
             ' Plots display Pc and NoPc Leg share of total net change variance (y-axis) vs Spreads net change cumulative distribution (x-axis).' +
             ' For current spreads we use settle data from ' + FromDate + ' to the most recent settle and' +
             ' for 2012-2014 spreads we are using settle data from ' + FromDate + ' to ' + ToDate + '. </p>'),
      BorderHtmlW, HBox([figure32, figure33]), BorderHtmlW])

To clarify matters, here is how we come up with these plots, step by step:

>- Aggregate daily net change data for consecutive spreads, we do this by building an equally weighted portfolio.
>
>
>- Repeat the process above but for their legs separatedly (equally weighted portfolio of Pc and NoPc legs).
>
>
>- Compute the cumulative distribution for spreads daily net changes (absolute values).
>
>
>- Compute PC and noPc leg net change variance corresponding to each quantile of the consecutive spreads daily net change cumulative distribution.
>
>
>- Compute total leg variance for each one of the quantiles mentioned above, as simply the sum of the Pc and NoPc leg net change variance. Then compute the share of that total variance corresponding to the Pc leg.
>
>
>- Plot the share of total daily net change variance corresponding to the Pc-leg vs the consecutive spreads aggregated daily net change data cumulative distribution (absolute values).

We can see how, for current spreads (2019 and 2020), the larger the daily moves the more relevance the PC-leg net change variance has. This is not the case for Pre-Hiking Cycle consecutive spreads, where leg share of total net change variance was more uniformly distributed between the Pc and NoPc legs. So yes, the short answer is that, at least for now, its safe to say that the PC leg is much more prevalent on the whole if we look at current spreads vs 2012-2014 spreads, and that PC-bias is still being taken into account.

### But...

We've stated in the previous section how there is a clear difference between 2019 and 2020 spreads, at least for now. Reasons stated above, we would expect 2020 spreads to behave way better, and have a more uniform behaviour for both their legs. Lets take a look at the data:

In [65]:
#Set of hike spreads 
Spreads_set = ['F20H20', 'J20M20', 'N20U20']

#From date
FromDate = '2019/01'

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData.index = SpreadD[3][FromDate:].index

#NET CHANGE
NetChangeSpreads_3 = GridFullData.diff()


##PC LEG
legs = ['H20', 'M20', 'U20']

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangePcLegs_3 = GridFullData_legs.diff()


##NO PC LEG
legs = ['F20', 'J20', 'N20']


#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangeNoPcLegs_3 = GridFullData_legs.diff()


############################################################
############################################################
############################################################


Portfolio_Spreads_3 = (NetChangeSpreads_3 * -2).dot(np.repeat(1/len(NetChangeSpreads_3.columns), 
                     len(NetChangeSpreads_3.columns)))

Portfolio_PcLegs_3 = NetChangePcLegs_3.dot(np.repeat(1/len(NetChangePcLegs_3.columns), 
                     len(NetChangePcLegs_3.columns)))

Portfolio_NoPcLegs_3 = NetChangeNoPcLegs_3.dot(np.repeat(1/len(NetChangeNoPcLegs_3.columns), 
                     len(NetChangeNoPcLegs_3.columns)))

Portfolio_Spreads_3 = abs(Portfolio_Spreads_3)

bins = 5
SpreadsHist_3, Spreads_edges_3 = np.histogram(Portfolio_Spreads_3[~np.isnan(Portfolio_Spreads_3)], bins = bins)

#VARIANCE
#Compute variance of the average PC and no PC legs, as per the cumulative distribution of the 
#average noPc-Pc spreads portfolio
VarianceOutDf_3 = pd.DataFrame([], index = ['Spreads Quantiles', 'PcLeg Variance', 
                               'NoPcLeg Variance', 'Total Variance', 
                               'PcLeg share Total Variance', 
                               'NoPcLeg share Total Variance'], 
                             columns = ['Q' + str(k) for k in np.arange(1/bins, 1.1, 1/bins)])

#Fill the data frame
for i in range(1, len(Spreads_edges_3)):
    
    ithEdgeIndices = Portfolio_Spreads_3[Portfolio_Spreads_3 <= Spreads_edges_3[i]].index
    
    #Variances
    ithEdge_Pc_var = Portfolio_PcLegs_3[ithEdgeIndices].var()
    ithEdge_NoPc_var = Portfolio_NoPcLegs_3[ithEdgeIndices].var()
    
    Total_var = ithEdge_Pc_var + ithEdge_NoPc_var 
    PcShare_var = ithEdge_Pc_var / Total_var
    NoPcShare_var = ithEdge_NoPc_var / Total_var    
    
    VarianceOutDf_3.iloc[0, i - 1] = Spreads_edges_3[i]
    VarianceOutDf_3.iloc[1, i - 1] = ithEdge_Pc_var
    VarianceOutDf_3.iloc[2, i - 1] = ithEdge_NoPc_var
    VarianceOutDf_3.iloc[3, i - 1] = Total_var
    VarianceOutDf_3.iloc[4, i - 1] = PcShare_var
    VarianceOutDf_3.iloc[5, i - 1] = NoPcShare_var


############################################################
############################################################
############################################################


#Set of hike spreads 
Spreads_set = ['K19M19', 'Q19U19'] 
               
#From date
FromDate = '2019/01'

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData.index = SpreadD[3][FromDate:].index

#NET CHANGE
NetChangeSpreads_4 = GridFullData.diff()


##PC LEG
legs = ['M19', 'U19'] 

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangePcLegs_4 = GridFullData_legs.diff()


##NO PC LEG
legs = ['K19', 'Q19']


#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangeNoPcLegs_4 = GridFullData_legs.diff()


############################################################
############################################################
############################################################


Portfolio_Spreads_4 = (NetChangeSpreads_4 * -2).dot(np.repeat(1/len(NetChangeSpreads_4.columns), 
                     len(NetChangeSpreads_4.columns)))

Portfolio_PcLegs_4 = NetChangePcLegs_4.dot(np.repeat(1/len(NetChangePcLegs_4.columns), 
                     len(NetChangePcLegs_4.columns)))

Portfolio_NoPcLegs_4 = NetChangeNoPcLegs_4.dot(np.repeat(1/len(NetChangeNoPcLegs_4.columns), 
                     len(NetChangeNoPcLegs_4.columns)))

Portfolio_Spreads_4 = abs(Portfolio_Spreads_4)

bins = 5
SpreadsHist_4, Spreads_edges_4 = np.histogram(Portfolio_Spreads_4[~np.isnan(Portfolio_Spreads_4)], bins = bins)

#VARIANCE
#Compute variance of the average PC and no PC legs, as per the cumulative distribution of the 
#average noPc-Pc spreads portfolio
VarianceOutDf_4 = pd.DataFrame([], index = ['Spreads Quantiles', 'PcLeg Variance', 
                               'NoPcLeg Variance', 'Total Variance', 
                               'PcLeg share Total Variance', 
                               'NoPcLeg share Total Variance'], 
                             columns = ['Q' + str(k) for k in np.arange(1/bins, 1.1, 1/bins)])

#Fill the data frame
for i in range(1, len(Spreads_edges_4)):
    
    ithEdgeIndices = Portfolio_Spreads_4[Portfolio_Spreads_4 <= Spreads_edges_4[i]].index
    
    #Variances
    ithEdge_Pc_var = Portfolio_PcLegs_4[ithEdgeIndices].var()
    ithEdge_NoPc_var = Portfolio_NoPcLegs_4[ithEdgeIndices].var()
    
    Total_var = ithEdge_Pc_var + ithEdge_NoPc_var 
    PcShare_var = ithEdge_Pc_var / Total_var
    NoPcShare_var = ithEdge_NoPc_var / Total_var    
    
    VarianceOutDf_4.iloc[0, i - 1] = Spreads_edges_4[i]
    VarianceOutDf_4.iloc[1, i - 1] = ithEdge_Pc_var
    VarianceOutDf_4.iloc[2, i - 1] = ithEdge_NoPc_var
    VarianceOutDf_4.iloc[3, i - 1] = Total_var
    VarianceOutDf_4.iloc[4, i - 1] = PcShare_var
    VarianceOutDf_4.iloc[5, i - 1] = NoPcShare_var


############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = np.reshape(np.array(NetChangeNoPcLegs.iloc[:, 2:].T), 
           (1, len(NetChangeNoPcLegs)*3))
y2_data = np.reshape(np.array(NetChangePcLegs.iloc[:, 2:].T), 
           (1, len(NetChangePcLegs)*3))


KDE_data = KDEforPlot(y_data[0][~np.isnan(y_data)[0]])
x3_data = KDE_data[0]
y3_data = KDE_data[1]

KDE2_data = KDEforPlot(y2_data[0][~np.isnan(y2_data)[0]])
x4_data = KDE2_data[0]
y4_data = KDE2_data[1]

#Labels (skew and kurtosis)
y_Stats = [round(skew(y_data[0][~np.isnan(y_data)[0]]), 2), 
           round(kurtosis(y_data[0][~np.isnan(y_data)[0]]), 2)]

y2_Stats = [round(skew(y2_data[0][~np.isnan(y2_data)[0]]), 2), 
            round(kurtosis(y2_data[0][~np.isnan(y2_data)[0]]), 2)]

DistrStat = ['Skewness', 'Kurtosis']
y_Stats_Str = [DistrStat[i] + ': ' + str(y_Stats[i]) for i in range(0, 2)]
y2_Stats_Str = [DistrStat[i] + ': ' + str(y2_Stats[i]) for i in range(0, 2)]
############################################################

#Step 1: Scales & Axes
############################################################     
#Scales
Hist_x = LinearScale()
Hist_y = LinearScale()

#X-Axis
Hist_ax_x = Axis(scale = Hist_x, 
                 tick_format = '.2f', 
                 tick_style = {'font-size': '12px'},                 
                 label_location = 'end',
                 label = 'Net Change (bps)',
                 label_offset = '40px')


#Y-Axis
Hist_ax_y = Axis(scale = Hist_y, 
                 orientation = 'vertical', 
                 tick_format = '.2f',     
                 tick_style = {'font-size': '12px'}, 
                 grid_color = '#EDEDED',                 
                 label_location = 'end',
                 label = 'Freq (normalized)',
                 label_offset = '40px')
############################################################    

#Step 2: Marks (plots)
############################################################
#Histogram plot (price)
HistPlot = Hist(sample = y_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 8,
                colors = ['#01B0F0'], 
                stroke = '#D8D8D8',
                labels = ['NoPc Leg NC'],
                display_legend = True)    

#Setting the tick values to be the mid points of the bins
Hist_ax_x.tick_values = HistPlot.midpoints

#Setting histogram bar opacities 
HistPlot.opacities = [0.4] * HistPlot.bins


#Histogram plot (price)
HistPlot2 = Hist(sample = y2_data, 
                normalized =  True,
                scales = {'sample': Hist_x, 
                          'count': Hist_y}, 
                padding = 0.25, 
                bins = 10,
                colors = ['#DE2369'], 
                stroke = '#D8D8D8',
                labels = ['Pc Leg NC'],
                display_legend = True)    

#Setting histogram bar opacities 
HistPlot2.opacities = [0.4] * HistPlot2.bins


#Density plots
lin = Lines(x = x3_data, y = y3_data, 
            scales = {'x': Hist_x, 'y' : Hist_y},
            colors = ['#0070C0'], 
            stroke_width = 3, interpolation = 'cardinal')

lin2 = Lines(x = x4_data, y = y4_data, 
            scales = {'x': Hist_x, 'y' : Hist_y},
            colors = ['#DE2369'], 
            stroke_width = 3, interpolation = 'cardinal')

LabelsPlot = Label(x = [0.03, 0.03], y = [0.5, 0.45], 
                   text = [(', ').join(y_Stats_Str), 
                           (', ').join(y2_Stats_Str)], 
                   default_size = 13,
                   font_weight = 'normal',
                   colors = ['#01B0F0', '#DE2369'], 
                   enable_move = True)
############################################################    

#Step 3: Hover interaction
############################################################      
#Creating the tooltip
def_tt = Tooltip(fields = ['count', 'midpoint'], 
         formats = ['.2f', '.2f'], 
         labels = ['Freq', 'Midpoint'])

#Set up the tooltip interaction (price)
HistPlot.tooltip = def_tt
HistPlot.interactions = {'hover': 'tooltip'}

#Set up the tooltip interaction (price)
HistPlot2.tooltip = def_tt
HistPlot2.interactions = {'hover': 'tooltip'}
############################################################

#Step 4: Figure (putting it all together)
############################################################         
#initial Figure
figure27 = Figure(marks = [HistPlot, HistPlot2, 
                           lin, lin2, LabelsPlot], 
                 axes = [Hist_ax_x, Hist_ax_y])

#Figure Title
figure27.title = ('Figure 37: ' + 
                  ' 2020 Spreads Aggregated Leg NC Data')

#Figure layout (except size attributes, defined on section 4)
figure27.title_style = {'font-size': '13px'}
figure27.background_style = {'fill': 'white'} 

figure27.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure27.layout.width = '45%'
figure27.layout.left = '30px'
figure27.layout.height = '450px'
figure27.legend_location = 'top-left'

############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = np.array([VarianceOutDf_2.iloc[4, :], 
                   VarianceOutDf_4.iloc[4, :],                     
                   VarianceOutDf_3.iloc[4, :]])

#Indices (X tick labels, days to maturity)
x_data = np.arange(1/bins, 1.1, 1/bins)
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_sc = LinearScale() 
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_sc, 
                   tick_style = {'font-size': '12px'}, 
                   tick_format = '0.2f',
                   grid_color = '#EDEDED', 
                   label_location = 'end',
                   label = 'Spreads Nc Cumulative Distribution',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.2f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED', 
                   label_location = 'end',
                   label = 'Pc-Leg Share of Total Variance',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['2013-2014 Spreads', 
                         '2019 Spreads', 
                         '2020 Spreads'], 
               scales = {'x': MainTs_x_sc, 
                         'y' : MainTs_y_sc},
               colors = ['#0070C0', '#01B0F0',  
                                    '#3CDBCC'], #, '#31DC8D'
               stroke_width = 4,
               marker = 'circle',
               interpolation = 'cardinal',
               marker_size = 120,
               visible = True, display_legend = True)
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['.2f', '.2f'], 
         labels = ['Spread Nc Quantile', 
                   'PcLeg Share of Variance'])

TsPlot.tooltip = def_tt
TsPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure34 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure34.title =  ('Figure 38: ' +
                  ' Consecutive Spreads Pc-Leg Share of Total NC Variance')

#Figure layout (except size attributes, defined on section 4)
figure34.title_style = {'font-size': '13px'}
figure34.background_style = {'fill': 'white'} 
    
figure34.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure34.layout.width = '45%'
figure34.layout.height = '450px'
figure34.legend_location = 'top-left'
figure34.layout.left = '60px'

In [66]:
VBox([widgets.HTML('<a id="Figure_37"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' Current and 2013-2014 Consecutive Hike Spreads Aggregated Leg Net Change Data.' + 
             ' First plot shows Pc and NoPc Leg histograms for 2020 consecutive spreads only.'      
             ' Second plot shows Pc-Leg share of total net change variance (y-axis) vs Spreads net change cumulative distribution (x-axis)'
             ' for 2019, 2020 and 2013-2014 spreads.' +
             ' For current spreads we use settle data from ' + FromDate + ' to the most recent settle and' +
             ' for 2012-2014 spreads we are using settle data from ' + FromDate + ' to ' + ToDate + '. </p>'),
      BorderHtmlW, HBox([figure27, figure34]), BorderHtmlW])

If we look at [`Figure 37`](#Figure_37), we can see that once we separate 2020 spreads from their 2019 peers, we can see both legs (the Pc and NoPc leg) behaving similarly. For 2020 spreads, both legs daily net change distributions are very much like the other, with similar tails and shape (skewness and kurtosis on the plot legend). 

We can also repeat the process we did for Figures 31 and 32, this time separating both the 2019 spreads set from the 2020 spreads one, plot the Pc-leg share of total variance vs the Spread's Net change cumulative distributions [`(Figure 38)`](#Figure_37) and clearly see the difference: For 2019 spreads, the larger the daily moves, on average, the larger the share of total leg net change variance attributed to the Pc-leg. The same cannot be said of 2020 spreads, as Pc-leg net change variance as share of the total stayed close to 50%. 

Finally, we include the same measure for the Pre-Hiking Cycle spreads, see how 2020 spreads appear more in line with these spreads. This may not mean anything yet, it only indicates that for now the curve in 2020 is smoother, and that both Quarterly and Non-Quarterly meetings have been moving more uniformly. 

## How Well Are We Hedged Then?

We jump to the second question of relevance regarding the trading of these spreads, as a result of the answers we found to the previous question. If our base case is that of a smooth curve where shorting opportunities may arise on these NoPc-Pc spreads, and, so far, the Pc-leg continues to have a stronger significance regarding the pricing of expected hike/cut estimations, with its volatility having a larger impact on the spread price: 

are we trading with a fragile hedge, completely exposed to the Quarterly-meeting leg? 

In [67]:
#Set of hike spreads 
Spreads_set = ['K19M19', 'Q19U19', 
               'F20H20', 'J20M20', 'N20U20']

#From date
FromDate = '2019/01'

GridFullData = pd.DataFrame(columns = Spreads_set)

#Get the spreads data (full set)
for i in range(0, len(Spreads_set)):

    Strategy = Spreads_set[i]

    #Spread data
    SpreadD = GetSspreadDLite(Strategy, Data = FFsDataList, 
                              Hikes = True) 

    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData.index = SpreadD[3][FromDate:].index

#NET CHANGE
NetChangeSpreads = GridFullData.diff()


##PC LEG
legs = ['M19', 'U19', 'H20', 'M20', 'U20']

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangePcLegs = GridFullData_legs.diff()


##NO PC LEG
legs = ['K19', 'Q19', 'F20', 'J20', 'N20']

#Get legs data and net changes
GridFullData_legs = pd.DataFrame(columns = legs)

for i in range(0, len(legs)):

    #Spread data
    SpreadD = GetSspreadDLite(legs[i], Data = FFsDataList, 
                              Hikes = True) 
    
    #Gridmap data
    ithGridDat = np.array(SpreadD[3][FromDate:]).T[0]
    
    #Populate dataframe
    GridFullData_legs.iloc[:, i] = ithGridDat
    
    #Date indices
    GridFullData_legs.index = SpreadD[3][FromDate:].index  
    

#NET CHANGE
NetChangeNoPcLegs = GridFullData_legs.diff()


############################################################
############################################################
############################################################


#Plot title
Title = 'How Well Are We Hedged?'

#Cumulative Net change data 
CumNc_Factors = FFsDataList['Factors_New_Data'][FromDate:].diff()

#Equally weighted portfolio (spreads)
Portfolio_Spreads = (NetChangeSpreads.cumsum() * -2).dot(np.repeat(1/len(NetChangeSpreads.columns), 
                                                len(NetChangeSpreads.columns)))

#Equally weighted portfolio (PC meetings alone)
Portfolio_PCHikes = NetChangePcLegs.dot(np.repeat(1/len(NetChangePcLegs.columns), 
                                                  len(NetChangePcLegs.columns))).cumsum()


############################################################
############################################################
############################################################


##DRAWDOWN
#Drawdown series
LevelDrawdown = (CumNc_Factors['PcLevel'].cumsum() - CumNc_Factors['PcLevel'].cumsum().cummax())
SpreadPfDrawdown = (Portfolio_Spreads - Portfolio_Spreads.cummax())
PcMeetingPfDrawdown = (Portfolio_PCHikes - Portfolio_PCHikes.cummax())

#Cost of going unhedged
CostOfNoHedge = PcMeetingPfDrawdown - SpreadPfDrawdown 


##CORRELATIONS
#Rolling window
RollingWindow = 10

#Correlations data frame
CorrDf = pd.DataFrame(np.array([CumNc_Factors['PcLevel'].cumsum(),
                                Portfolio_Spreads, Portfolio_PCHikes]).T, 
             columns = ['PcLevel', 'Spreads', 'PcHikes'], 
             index = Portfolio_Spreads.index)

CorrDf_Normalized = (CorrDf - CorrDf.mean()) / CorrDf.std()

#Full period correlation 
HedgeCorr_PC = (CorrDf.iloc[:, 0].corr(CorrDf.iloc[:, 1]), 
                CorrDf.iloc[:, 0].corr(CorrDf.iloc[:, 2]))

#Rolling correlation series
RollingCorr_1 = CorrDf.iloc[:, 0].rolling(RollingWindow).corr(CorrDf.iloc[:, 1])
RollingCorr_2 = CorrDf.iloc[:, 0].rolling(RollingWindow).corr(CorrDf.iloc[:, 2])


############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = np.array([SpreadPfDrawdown, PcMeetingPfDrawdown])

#Indices (X tick labels, days to maturity)
x_data = Portfolio_Spreads.index
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = DateScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Date',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Drawdown (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['NoPc-Pc Spreads', 
                         'Pc Meetings (no hedge)',
                        'PcLevel'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#01B0F0', '#DE2369', 'black'], 
               stroke_width = 2,
                marker = 'circle', marker_size = 60, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['%d/%m/%Y', '.2f'], 
         labels = ['Date', 'Drawdown'])

TsPlot.tooltip = def_tt
TsPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure28 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure28.title = ('Figure 39: ' + 
                  ' Drawdown (bps) - Base: ' 
                  + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure28.title_style = {'font-size': '13px'}
figure28.background_style = {'fill': 'white'}
    
figure28.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure28.layout.width = '47%'
figure28.layout.height = '400px'
figure28.layout.left = '20px'
figure28.legend_location = 'bottom-left'


############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = np.array(CostOfNoHedge)

#Indices (X tick labels, days to maturity)
x_data = Portfolio_Spreads.index
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = DateScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Date',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Drawdown (bps)',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
BarPlot = Bars(x = x_data, 
               y = y_data, 
               labels = ['Drawdown Spread'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#01B0F0'],
               stroke = 'white', 
               visible = True, 
               display_legend = True, 
               padding = 0.4)
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['%d/%m/%Y', '.2f'], 
         labels = ['Date', 'Drawdown'])

BarPlot.tooltip = def_tt
BarPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure29 = Figure(marks = [BarPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure29.title = ('Figure 40: ' + 
                  ' Cost of Going Unhedged (Drawdown spread, in bps) - Base: ' 
                  + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure29.title_style = {'font-size': '13px'}
figure29.background_style = {'fill': 'white'}
    
figure29.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure29.layout.width = '48%'
figure29.layout.height = '400px'
figure29.layout.left = '40px'
figure29.legend_location = 'bottom-left'

### Two Portfolios

First of all, we need to define what we understand to be the hedge in this trade. we already said that we are ideally looking to short the Non-Quarterly meeting leg, as a hedge for our longs on the Quarterly-meeting leg. So, this means we are getting exposure to the general level of the Quarterly meeting's curve, while hedging it with the level of the Non-Quarterly meeting's curve. 

That out of the way, we can gauge the effectiveness of the hedge on these spreads by setting up two portfolios and comparing them:

>- The first one is an equally-weighted Portfolio composed of the hike spreads we are looking at currently, both for 2019 and 2020: K19M19, Q19U19, F20H20, J20M20 and N20U20 (long the Pc meetings and short the NoPc meetings).
>
>
>- The second one is an equally-weighted version that only has the relevant Quarterly-meetings: M19, U19, H20, M20 and U20. So, no hedge here (long the Pc meetings only). 
>
>

### Drawdown

one way of looking at how effective the hedge was since the Christmas rally (remember we are looking at the current curve "regime", where the market is pricing a Rate-cut cycle) is to look at both portfolio's respective drawdowns over the sample period.

Before we go on, we should establish what we define as drawdown: The portfolio's drawdown measures the lowest point of the portfolio (in bps) after its most recent high-water mark, in other words, the difference between the peak of the series and every value below it (every time the portfolio makes a new high the drawdown is zero). The drawdown is an useful indicator that allows us to see how the portfolio losses behave since the most recent peak, always assuming the portfolio remains unchanged. 

We can see both portfolio's respective drawdowns below [`(Figure 39)`](#Figure_39):

In [68]:
VBox([widgets.HTML('<a id="Figure_39"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' Current Consecutive hike spreads aggregated drawdown data.' + 
             ' First plot shows the drawdown of Consecutive spreads and Pc meetings with no hedge.'      
             ' Second plot shows the spread between both drawdown series, or the "cost" of not hedging the Pc level exposure.' +
             ' Settle data from ' + FromDate + ' to the most recent settle. </p>'),
      BorderHtmlW, HBox([figure28, figure29]), BorderHtmlW])

Both portfolio drawdowns are similar until we get to March, where we experienced an abrupt break for 2019 spreads, with a dovish Fed (March FOMC meeting on the 20th) and weak Euro area PMI data. If we compute the spread between both drawdowns [`(Figure 40)`](#Figure_39), we can refer to it as the cost of going un-hedged, we can see that the hedge does become useful during March, where the cost of going naked long on Pc-meetings averaged 3 basis points on the worst week (last week of March).

Some initial insights can be drawn from this data:

>- We know Non-Quarterly meetings tend to remain very steady when there is no strong market volatility, from January through the first week of March, the market was calm and we can observe that there was little to no difference, on average, between hedging the Pc-leg exposure and not doing so. 
>
>
>- Once the market reacted to what was perceived as bad data for US rates (March), the hedge becomes more useful, as in spite of the market pricing cuts more aggressively on Quarterly meetings, Non-Quarterly meetings saw moves on the same direction. This is one point in favor of the hedge clearly, we want it to work, at least partially, when the market gets into a panic. 
>
>

In [69]:
#Plot data
############################################################
y_data = np.array(CorrDf_Normalized.T)

#Indices (X tick labels, days to maturity)
x_data = Portfolio_Spreads.index
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = DateScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Date',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Normalized Series',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['PcLevel',  
                         'NoPc-Pc Spreads', 
                         'Pc Meetings (no hedge)'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#3CDBCC', '#01B0F0', 
                         '#DE2369'], #'#3CDBCC', '#31DC8D'
               stroke_width = 2,
               marker = 'circle', 
               marker_size = 40, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['%d/%m/%Y', '.2f'], 
         labels = ['Date', 'Correlation'])

TsPlot.tooltip = def_tt
TsPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure30 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure30.title =  ('Figure 41: ' +
                  ' Normalized Time Series - Base: ' 
                  + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure30.title_style = {'font-size': '13px'}
figure30.background_style = {'fill': 'white'}
    
figure30.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure30.layout.width = '47%'
figure30.layout.height = '400px'
figure30.layout.left = '20px'
figure30.legend_location = 'bottom-left'


############################################################
############################################################
############################################################


#Plot data
############################################################
y_data = np.array([RollingCorr_1, RollingCorr_2])

#Indices (X tick labels, days to maturity)
x_data = Portfolio_Spreads.index
############################################################

#Step 1: Scales & Axes
############################################################ 
#Scales 
MainTs_x_dates = DateScale()
MainTs_y_sc  = LinearScale() 

#X-Axis
MainTs_x_ax = Axis(scale = MainTs_x_dates, 
                   tick_style = {'font-size': '12px'}, 
                   num_ticks = 10,
                   label_location = 'end',
                   label = 'Date',
                   label_offset = '40px')

#Y-Axis
MainTs_y_ax = Axis(scale = MainTs_y_sc, 
                   orientation = 'vertical', 
                   tick_format = '0.1f', 
                   tick_style = {'font-size': '12px'},
                   grid_color = '#EDEDED',
                   label_location = 'end',
                   label = 'Correlation',
                   label_offset = '40px')
############################################################    
 
#Step 2: Marks (plots)
############################################################
#Factors main time series 
TsPlot = Lines(x = x_data, 
               y = y_data, 
               labels = ['NoPc-Pc Spreads', 
                         'Pc Meetings (no hedge)'], 
               scales = {'x': MainTs_x_dates, 
                         'y' : MainTs_y_sc},
               colors = ['#01B0F0', '#DE2369'], 
               stroke_width = 2,
               marker = 'circle', 
               marker_size = 40, 
               visible = True, 
               display_legend = True)
############################################################

#Step 3: Hover interaction
############################################################        
#Creating the tooltip
def_tt = Tooltip(fields = ['x', 'y'], 
         formats = ['%d/%m/%Y', '.2f'], 
         labels = ['Date', 'Correlation'])

TsPlot.tooltip = def_tt
TsPlot.interactions = {'hover': 'tooltip'}    
############################################################ 


#Step 3: Figure (putting it all together)
############################################################     
#initial Figure
figure31 = Figure(marks = [TsPlot], 
                 axes = [MainTs_x_ax, MainTs_y_ax])

#Figure Title
figure31.title = ('Figure 42: ' + 
                  ' PcLevel ' + str(RollingWindow) + 
                  '-Day Rolling Correlation - Base: ' 
                  + FromDate)

#Figure layout (except size attributes, defined on section 4)
figure31.title_style = {'font-size': '13px'}
figure31.background_style = {'fill': 'white'}
    
figure31.fig_margin = {'top':40, 'bottom':50, 'left':60, 'right':20}
figure31.layout.width = '48%'
figure31.layout.height = '400px'
figure31.layout.left = '40px'
figure31.legend_location = 'bottom-left'

### Pc Curve Level Correlation

Another approach to judging the effectivenes of the hedge is to look at the relationship between both portfolios and the Press Conference hikes level factor. We can do this by looking at their normalized time series and rolling correlation between both portfolios time series and the Press Conference level. Normalizing their time series lets us compare their variabilities, eliminating scaling problems, while the rolling correlation can help us get a hint on whether the hedge actually works on periods of stress (the higher the correlation, the less effective the hedge is).

In [70]:
VBox([widgets.HTML('<a id="Figure_41"></a>'),
      widgets.HTML('<p style = "color: #125CB2;"> <b>' + Title + ':</b>' + 
             ' Current Consecutive hike spreads, Pc hikes aggregated data and PcLevel factor data.' + 
             ' Normalized time series and 10-day rolling correlation for both portfolios vs the PcLevel factor.'      
             ' Settle data from ' + FromDate + ' to the most recent settle. </p>'),
      BorderHtmlW, HBox([figure30, figure31]), BorderHtmlW])

Both portfolio's variability were highly correlated to the level of the Press Conference hikes curve, still, we can see periods where the hedged portfolio diverged [`(Figure 41)`](#Figure_41). Since we are basically trading both curve's levels on these type of spreads (Pc vs NoPc level, as a hedge), the fact that we can see divergences does present an opportunity for trading.

Given what we've seen so far on this document, seeing how both portfolio's variabilities over the sample period are highly correlated to the Pc-hikes level factor is to be expected. On the surface we see that the hedge is not perfect, but, when we look at the rolling correlation time series on [`Figure 42`](#Figure_41), we can see that the consecutive spreads portfolio does hold better on stress situations such as the March rally on rates. Particularly on the worst day (March 27th), we can see how the correlation goes down to negative, this likely due to the way 2020 spreads behaved during that week. 

Still, the NoPc-Pc spreads portfolio correlation to the factor is high across the whole sample period, at times, more than 80% of the variability of the portfolio is related to the level of the press conference curve, and that means that outside stress events the hedge was not perfect.